In [33]:
# Code to generate objective function Jacobian matrices from solution models for local minimization
# it also generates additional set of inequality constraints

# last update 24.02.23, NR
import numpy as np
import math as math
import cmath
import time
import random
from sympy import *
from sympy.printing import print_ccode

# EXPORT
import os, re
init_printing()

In [34]:
# non linear phase solver, include option to generate ghost points
def func_bnd_guess(name,bulk):
    eps = 1e-14
    if   name == 'bi':
        # def bnd():
        t   = 1
        f   = 1

        if bulk[7] == 0:
            t  = eps
        if bulk[8] == 0:
            f  = eps

        box_bounds = ((0, 1), (0, 1),(0, 1), (0, t),(0, f))
        iguess = [0.15, 0.25, 0.4, 0.17, 0.25]
            
    elif name == 'ksp':
        # def bnd():
        box_bounds = ((0, 1), (0, 1))
        iguess = [0.1, 0.001]
        
    elif name == 'g':
        # def bnd():
        f    = 1
        cr   = 1
        t    = 1

        if bulk[8] == 0:
            f  = eps
        if bulk[9] == 0:
            cr  = eps
        if bulk[7] == 0:
            t  = eps

        box_bounds = ((0, 1), (0, 1),(0, f), (0, cr),(0, t))
        iguess = [0.3, 0.2, 0.01, 0.01, 0.001]

    elif name == 'ep':
        # def bnd():
        f   = 1
        Q   = 1

        if bulk[8] == 0:
            f  = eps
            Q  = eps
        box_bounds = ((0, f), (0, Q))
        iguess = [0.8, 0.03]
        
    elif name == 'pl':
        # def bnd():
        box_bounds = ((0, 1), (0, 1))
        iguess = [0.8, 0.03]
       
        
    elif name == 'mu':
        # def bnd():
        f   = 1

        if bulk[8] == 0:
            f  = eps
            
        box_bounds = ((0, 1), (0, 1),(0, f), (0, 1),(0, 1))
        iguess = [0.25, 0.6, 0.17, 0.06, 0.004]
       
    elif name == 'ol':
        # def bnd():
        box_bounds = ((0, 1), (0, 1),(0, 1))
        iguess = [0.4, 0.002, 0.01]
            
    elif name == 'cd':
        # def bnd():
        h   = 1

        if bulk[10] == 0:
            h  = eps
        box_bounds = ((0, 1), (0, h))
        iguess = [0.8, 0.03]

    elif name == 'opx':
        # def bnd():
        f    = 1
        cr   = 1
        t    = 1

        if bulk[8] == 0:
            f  = eps
        if bulk[9] == 0:
            cr  = eps
        if bulk[7] == 0:
            t  = eps

        box_bounds = ((0, 1), (0, 2),(0, 1), (-1, 1),(0, f),(0, t), (0, cr),(0, 1))
        iguess = [0.05, 0.006, 0.025, 0.032, 0.1, 0.001, 0.001, 0.001] 

            
    elif name == 'cpx':
        # def bnd():
        f    = 1
        cr   = 1
        t    = 1

        if bulk[8] == 0:
            f  = eps
        if bulk[9] == 0:
            cr  = eps
        if bulk[7] == 0:
            t  = eps

        box_bounds = ((0, 1), (0, 2),(0, 1), (0, 1),(-1, 1),(0, f), (0, cr),(0, t), (0, 1))
        iguess = [0.075, 0.1120, 0.05, 0.11, -0.0005, 0.001, 0.001, 0.001, 0.001]

            
    elif name == 'spn':
        # def bnd():
        y   = 1
        c   = 1
        t   = 1

        if bulk[7] == 0:
            t  = eps
        if bulk[8] == 0:
            y  = eps
        if bulk[9] == 0:
            c  = eps_
        box_bounds = ((0, 1), (0, y),(0, c), (0, t),(-1, 1),(-1, 1), (-1, 1))
        iguess = [0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.02]
            
    elif name == 'hb':
        # def bnd():
        f  = 1
        t  = 1

        if bulk[7] == 0:
            t  = eps
        if bulk[8] == 0:
            f  = eps

        box_bounds = ((0, 1), (0, 1),(0, 1), (0, 1),(0, 1),(0, 1), (0, f),(0, t), (-1, 1),(-1, 1))
        iguess = [0.3, 0.2, 0.01, 0.45, 0.01, 0.8, 0.05, 0.01, -0.01, 0.1]
            
    elif name == 'liq':
        # def bnd():
        hm  = 1
        ek  = 1
        ti  = 1
        h2o = 1
        
        if bulk[7] == 0:#ti
            ti   = eps
        if bulk[8] == 0:#chr
            hm  = eps
        if bulk[9] == 0:#o
            ek  = eps
        if bulk[10] == 0:#h2o
            h2o = eps

        box_bounds = ((0, 1), (0, 1),(0, 1), (0, 1),(0, 1), (0, hm),(0, ek), (0, ti),(0, 1), (0, 1), (0, h2o))
        iguess = [0.2,0.2,0.1,0.1,0.05,0.001,0.001,0.001,0.001,0.001,0.001]

    elif name == 'ilm':
        # def bnd():
        x2  = 0
        if bulk[8] == 0:
            x2  = 1 - eps

        box_bounds = ((x2, 1.), (-0.99, 0.99))
        iguess = [0.8, 0.55]

    else:
        display('Either phase is not defined or there is a typo')
  

    return (box_bounds,iguess)

def digit2index2(eq):  
    eq =  re.sub(r'[m][u][z]([0-9][0-9])', r'mu[\1]',str(eq) )
    eq =  re.sub(r'[m][u][_][G][e][x]([0-9][0-9])', r'mu_Gex[\1]',str(eq) )
    eq =  re.sub(r'[p][h][i]([0-9][0-9])', r'phi[\1]',str(eq) )
    eq =  re.sub(r'[i][d][m]([0-9][0-9])', r'idm[\1]',str(eq) )
    eq =  re.sub(r'[g][b]([0-9][0-9])', r'gb[\1]',str(eq) )
    eq =  re.sub(r'[p]([0-9][0-9])', r'p[\1]',str(eq) )
    eq =  re.sub(r'[b][u][l][k]([0-9][0-9])', r'bulk[\1]',str(eq) )   
    eq =  re.sub(r'[c][o][m][p]([0-9][0-9])', r'chem_comp[\1]',str(eq) )   
    
    eq =  re.sub(r'[m][u][z]([0-9])', r'mu[\1]',str(eq) )  
    eq =  re.sub(r'[m][u][_][G][e][x]([0-9])', r'mu_Gex[\1]',str(eq) )
    eq =  re.sub(r'[p][h][i]([0-9])', r'phi[\1]',str(eq) )
    eq =  re.sub(r'[i][d][m]([0-9])', r'idm[\1]',str(eq) )
    eq =  re.sub(r'[g][b]([0-9])', r'gb[\1]',str(eq) )
    eq =  re.sub(r'[p]([0-9])', r'p[\1]',str(eq) )
    eq =  re.sub(r'[c][o][m][p]([0-9])', r'chem_comp[\1]',str(eq) )
    eq =  re.sub(r'[b][u][l][k]([0-9])', r'bulk[\1]',str(eq) )  

    eq =  re.sub(r'[l][o][g]', r'creal(clog',str(eq) )
    eq =  re.sub(r'[)][ ][+][ ][g]', r')) + g',str(eq) )
    eq =  re.sub(r'[p][o][w]', r'cpow',str(eq) )
    return(eq) 

def replaceSymbols(op,in_var,out_var):    
    regin  = 'r\'([' + in_var   +']*)\''
    regout = 'r\''  + out_var  +'\''
    t0 =  re.sub(in_var, out_var,op )  
    return(t0)
# for make endmembers -> get gb
def print_em_gb(em,din):
    dout = str(din)
    for i in din.free_symbols:
        if i in em:
            if (str(i)[0:2] == 'd_'):
                sym = str(i)[2:]+'_di.gb'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'o_'):
                sym = str(i)[2:]+'_or.gb'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'e_'):
                sym = str(i)[2:]+'_eq.gb'
                dout= replaceSymbols(dout,str(i),sym)
            else:
                sym = str(i)+'_eq.gb'
                dout= replaceSymbols(dout,str(i),sym)
        else:
                sym = 'z_b.'+str(i)
                dout= replaceSymbols(dout,str(i),sym)
            
    return dout
 
def print_em_shearM(em,din):
    din_dependent = sympify('0');
    for i in din.free_symbols: 
        if i in em:
            r = din.as_independent(i)
    #             print(sympify(r[1]))
            din_dependent += (sympify(r[1]))

    dout = str(din_dependent)
    for i in din.free_symbols:
        if i in em:
            if (str(i)[0:2] == 'd_'):
                sym = str(i)[2:]+'_di.ElShearMod'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'o_'):
                sym = str(i)[2:]+'_or.ElShearMod'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'e_'):
                sym = str(i)[2:]+'_eq.ElShearMod'
                dout= replaceSymbols(dout,str(i),sym)
            else:
                sym = str(i)+'_eq.ElShearMod'
                dout= replaceSymbols(dout,str(i),sym)

    return dout

def print_em_C(em,din):
    din_dependent = sympify('0');
    for i in din.free_symbols: 
        if i in em:
            r = din.as_independent(i)
            din_dependent += (sympify(r[1]))

    dout = str(din_dependent)
    for i in din.free_symbols:
        if i in em:
            if (str(i)[0:2] == 'd_'):
                sym = str(i)[2:]+'_di.C[i]'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'o_'):
                sym = str(i)[2:]+'_or.C[i]'
                dout= replaceSymbols(dout,str(i),sym)
            elif (str(i)[0:2] == 'e_'):
                sym = str(i)[2:]+'_eq.C[i]'
                dout= replaceSymbols(dout,str(i),sym)
            else:
                sym = str(i)+'_eq.C[i]'
                dout= replaceSymbols(dout,str(i),sym)
    return dout

def print_W(w):
    var = str(W[i])
    var = replaceSymbols(var,'P','SS_ref_db.P')
    var = replaceSymbols(var,'T','SS_ref_db.T')
    return var

def eval_sf(sf_pc,cv2sf):
    sf_ok = 1
    for  i,val in enumerate(sf):
        sf_pc[i] = eval(cv2sf[i])  
        if sf_pc[i] < 1e-6:
            sf_ok = 0
    return sf_ok
op = ''

In [35]:
# ss database igneous _ig

def liq_ig():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 12

    W         = [0]*66
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    psf       = [0]*n_em; 
    idm       = [0]*n_em;
    stp       = 0.19;
    # Initial guess -> ca,k
    IG       = [0]*(n_em-1);

    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # wo  range
    Range[1]  = [0,1]; # sl  range
    Range[2]  = [0,1]; # fo  range
    Range[3]  = [0,1]; # fa  range
    Range[4]  = [0,1]; # jd  range
    Range[5]  = [0,1]; # hm  range
    Range[6]  = [0,1]; # ek  range
    Range[7]  = [0,1]; # ti  range
    Range[8]  = [0,1]; # kj  range
    Range[9]  = [0,1]; # yct range
    Range[10] = [0,1]; # h2o range

    emname  = ['q4L','sl1L','wo1L','fo2L','fa2L','jdL','hmL','ekL','tiL','kjL','ctL','h2o1L']
    make    = [0]*n_em;
    make[0] = sympify('4* qL + 0.22 - 0.059* P');
    make[1] = sympify('silL + 6.2 - 0.318 *P');
    make[2] = sympify('woL - 0.45 - 0.114 *P');
    make[3] = sympify('2 *foL + 8.67 - 0.131 *P');
    make[4] = sympify('2* faL + 13.7 - 0.055 *P');
    make[5] = sympify('abL - qL + 12.19 - 0.089 *P');
    make[6] = sympify('1/2* hemL + 3.3 - 0.032* P');
    make[7] = sympify('1/2 *eskL + 24.85 + 0.245 *P');
    make[8] = sympify('ruL + 5.58 - 0.489* P');
    make[9] = sympify('kspL - qL + 11.98 - 0.21* P');
    make[10] = sympify('woL + silL - qL - 108.3 + 0.055* T + 0.053 *P');
    make[11] = sympify('h2oL + 3.2 - 0.0039 *T + 0.00087 *P');

    
    p[0] = sympify('1 - wo - sl - jd - fa - fo - hm - ek - ti - kj - h2o + 1/4*yct*(4 - 3*ek - 3*fa - 3*fo - 3*hm - 3*jd - 3*kj - 3*sl - 3*ti - 3*wo - 3*h2o)');
    p[1] = sympify('sl + 3/4*yct*sl + (-yct)');
    p[2] = sympify('wo + 3/4*yct*wo + (-yct)');
    p[3] = sympify('fo + 3/4*yct*fo');
    p[4] = sympify('fa + 3/4*yct*fa');
    p[5] = sympify('jd + 3/4*yct*jd');
    p[6] = sympify('hm + 3/4*yct*hm');
    p[7] = sympify('ek + 3/4*yct*ek');
    p[8] = sympify('ti + 3/4*yct*ti');
    p[9] = sympify('kj + 3/4*yct*kj');
    p[10] = sympify('yct');
    p[11] = sympify('h2o + 3/4*yct*h2o');
    
    
    #     Site fractions
    pq   = sympify('1 - wo - sl - jd - fa - fo - hm - ek - ti - kj - h2o + 1/4*yct*(4 - 3*ek - 3*fa - 3*fo - 3*hm - 3*jd - 3*kj - 3*sl - 3*ti - 3*wo - 3*h2o)');
    psl  = sympify('sl + 3/4*yct*sl + (-yct)');
    pwo  = sympify('wo + 3/4*yct*wo + (-yct)');
    pjd  = sympify('jd + 3/4*yct*jd');
    phm  = sympify('hm + 3/4*yct*hm');
    pek  = sympify('ek + 3/4*yct*ek');
    pti  = sympify('ti + 3/4*yct*ti');
    pkj  = sympify('kj + 3/4*yct*kj');
    pct  = sympify('yct');
    pol  = sympify('fo + fa + 3/4*yct*(fo + fa)');
    sumT = sympify('1 - h2o + (-3/4*yct)*h2o');
    mgM  = sympify('4*fo');
    feM  = sympify('4*fa');
    CaM  = sympify('wo');
    AlM  = sympify('sl');
    sumM = sympify('4*fo + 4*fa + sl + wo');
    xh   = sympify('h2o');
    xv   = sympify('1 - h2o');

    sf = [pq, psl, pwo, pjd, phm, pek, pti, pkj, pct, pol, sumT, mgM, feM, CaM, AlM, sumM, xh, xv];
  
    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 9.5 - 0.10*P;
    W[1] = -10.3;
    W[2] = -26.5 - 3.12*P;
    W[3] = -12.0 - 0.55*P;
    W[4] = -15.1 - 0.13*P;
    W[5] = 20.0;
    W[6] = 0;
    W[7] = 24.6;
    W[8] = -17.8 - 0.05*P;
    W[9] = -14.6;
    W[10] = 17.8 - 0.61*P;
    W[11] = -26.5 + 0.85*P;
    W[12] = 2.2;
    W[13] = 2.5;
    W[14] = 16.8;
    W[15] = -5.0;
    W[16] = 0;
    W[17] = 15.2 - 0.04*P;
    W[18] = 7.0;
    W[19] = 4.0;
    W[20] = 23.7 - 0.94*P;
    W[21] = 25.5 + 0.11*P;
    W[22] = 14.0;
    W[23] = -1.2;
    W[24] = 0;
    W[25] = 0;
    W[26] = 18.0;
    W[27] = -1.1;
    W[28] = 9.5;
    W[29] = 40.3 - 0.86*P;
    W[30] = 18.0;
    W[31] = 1.5;
    W[32] = 0;
    W[33] = 0;
    W[34] = 7.5;
    W[35] = 3.0;
    W[36] = -5.6;
    W[37] = 9.4 - 1.58*P;
    W[38] = 7.5 - 0.05*P;
    W[39] = -30.0;
    W[40] = 0;
    W[41] = 6.7;
    W[42] = 10.0;
    W[43] = -6.5;
    W[44] = 9.2 - 1.58*P;
    W[45] = 10.0;
    W[46] = 0;
    W[47] = 16.5 + 0.14*P;
    W[48] = -5.9;
    W[49] = 7.6;
    W[50] = -8.3 - 0.06*P; 
    W[51] = 0;
    W[52] = 0;
    W[53] = 10.0;
    W[54] = 0;
    W[55] = 60.0 - 0.66*P;
    W[56] = 0;
    W[57] = 0;
    W[58] = 0;
    W[59] = 30.0 - 0.66*P; 
    W[60] = 9.0;
    W[61] = 0;
    W[62] = 30.0 - 0.60*P;
    W[63] = -5.6;
    W[64] = -0.1 + 0.22*P;
    W[65] = 17.3 + 0.05*P;

    v[0] = 100.00;
    v[1] = 120.00;
    v[2] = 140.00;
    v[3] = 240.00;
    v[4] = 100.00;
    v[5] = 120.00;
    v[6] = 100.00;
    v[7] = 100.00;
    v[8] = 100.00;
    v[9] = 100.00;
    v[10] = 100.00;
    v[11] = 100.00;

    idm[0]  =   sympify('xv**2*(sumT**-1)*pq');
    idm[1]  =   sympify('xv**2*(sumT**-1)*psl*AlM*sumM**-1');
    idm[2]  =   sympify('xv**2*(sumT**-1)*pwo*CaM*sumM**-1');
    idm[3]  =   sympify('xv**2*(sumT**-1)*pol*mgM**4*sumM**-4');
    idm[4]  =   sympify('xv**2*(sumT**-1)*pol*feM**4*sumM**-4');
    idm[5]  =   sympify('xv**2*(sumT**-1)*pjd');
    idm[6]  =   sympify('xv**2*(sumT**-1)*phm');
    idm[7]  =   sympify('xv**2*(sumT**-1)*pek');
    idm[8]  =   sympify('xv**2*(sumT**-1)*pti');
    idm[9]  =   sympify('xv**2*(sumT**-1)*pkj');
    idm[10] =   sympify('xv**2*(sumT**-1)*pct');
    idm[11] =   sympify('xh**2');
    
    sym_sf = ['pq', 'psl', 'pwo', 'pjd', 'phm', 'pek', 'pti', 'pkj', 'pct', 'pol', 'sumT', 'mgM', 'feM', 'CaM', 'AlM', 'sumM', 'xh', 'xv'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');
    gbase[6]  = sympify('gb6');
    gbase[7]  = sympify('gb7');
    gbase[8]  = sympify('gb8');
    gbase[9]  = sympify('gb9');
    gbase[10] = sympify('gb10');
    gbase[11] = sympify('gb11');


    symb     = ['wo','sl','fo','fa','jd','hm','ek','ti','kj','yct','h2o']
    
    in_var  = ['wo','sl','fo','fa','jd','hm','ek','ti','kj','yct','h2o','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','x[7]','x[8]','x[9]','x[10]','ln']
    in_sf = ['pq', 'psl', 'pwo', 'pjd', 'phm', 'pek', 'pti', 'pkj', 'pct', 'pol', 'sumT', 'mgM', 'feM', 'CaM', 'AlM', 'sumM', 'xh', 'xv'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]','sf[10]','sf[11]','sf[12]','sf[13]','sf[14]','sf[15]','sf[16]','sf[17]']

    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def fl_ig():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 11

    W         = [0]*55
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.99;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # wo  range
    Range[1]  = [0,1]; # sl  range
    Range[2]  = [0,1]; # fo  range
    Range[3]  = [0,1]; # fa  range
    Range[4]  = [0,1]; # jd  range
    Range[5]  = [0,1]; # hm  range
    Range[6]  = [0,1]; # ek  range
    Range[7]  = [0,1]; # ti  range
    Range[8]  = [0,1]; # kj  range
    Range[9]  = [0,1]; # h2o range
    
    emname  = ['qfL','slfL','wofL','fofL','fafL','jdfL','hmfL','ekfL','tifL','kjfL','h2o']
    make    = [0]*n_em;
    make[0] = sympify('4* qL + 2.1 - 0.051* P');
    make[1] = sympify('silL + 6.72 - 0.313 *P');
    make[2] = sympify('woL + 0.22 - 0.12 *P');
    make[3] = sympify('2* foL + 8.59 - 0.136* P');
    make[4] = sympify('2 *faL + 13.56 - 0.052 *P');
    make[5] = sympify('abL - qL + 12.32 - 0.099 *P');
    make[6] = sympify('1/2 *hemL + 4.05 - 0.077 *P');
    make[7] = sympify('1/2 *eskL + 24.75 + 0.245 *P');
    make[8] = sympify('ruL + 5.6 - 0.489 *P');
    make[9] = sympify('kspL - qL + 12.88 - 0.227 *P');
    make[10] = sympify('H2O');

    p[0] = sympify('1 - wo - sl - fo - fa - jd - hm - ek - ti - kj - h2o');
    p[1] = sympify('sl');
    p[2] = sympify('wo');
    p[3] = sympify('fo');
    p[4] = sympify('fa');
    p[5] = sympify('jd');
    p[6] = sympify('hm');
    p[7] = sympify('ek');
    p[8] = sympify('ti');
    p[9] = sympify('kj');
    p[10] = sympify('h2o');

    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 0;
    W[1] = 0;
    W[2] = 0;
    W[3] = 0;
    W[4] = 0;
    W[5] = 0;
    W[6] = 0;
    W[7] = 0;
    W[8] = 0;
    W[9] = 59.0 - 0.82*P;
    W[10] = 0;
    W[11] = 0;
    W[12] = 0;
    W[13] = 0;
    W[14] = 0;
    W[15] = 0;
    W[16] = 0;
    W[17] = 0;
    W[18] = 57.6 - 0.80*P;
    W[19] = 0;
    W[20] = 0;
    W[21] = 0;
    W[22] = 0;
    W[23] = 0;
    W[24] = 0;
    W[25] = 0;
    W[26] = 72.2 - 0.67*P;
    W[27] = 0;
    W[28] = 0;
    W[29] = 0;
    W[30] = 0;
    W[31] = 0;
    W[32] = 0;
    W[33] = 71.7 - 1.10*P;
    W[34] = 0;
    W[35] = 0;
    W[36] = 0;
    W[37] = 0;
    W[38] = 0;
    W[39] = 71.7 - 1.10*P;
    W[40] = 0;
    W[41] = 0;
    W[42] = 0;
    W[43] = 0;
    W[44] = 57.0 - 0.79*P;
    W[45] = 0;
    W[46] = 0;
    W[47] = 0;
    W[48] = 73.0 - 0.66*P;
    W[49] = 0;
    W[50] = 0;
    W[51] = 73.0 - 0.66*P;
    W[52] = 0;
    W[53] = 75.0 - 0.67*P;
    W[54] = 44.9 - 1.19*P;
    v = 0.0;

    #     Site fractions
    pq  = sympify('1 - wo - sl - fo - fa - jd - hm - ek - ti - kj - h2o');
    psl = sympify('sl');
    pwo = sympify('wo');
    pfo = sympify('fo');
    pfa = sympify('fa');
    pjd = sympify('jd');
    phm = sympify('hm');
    pek = sympify('ek');
    pti = sympify('ti');
    pkj = sympify('kj');
    ph2o= sympify('h2o');
    fac = sympify('1 - h2o');

    sf = [pq, psl, pwo, pfo, pfa, pjd, phm, pek, pti, pkj, ph2o, fac];

    #     Ideal mixing parameters
    idm[0]  =   sympify('fac*pq');
    idm[1]  =   sympify('fac*psl');
    idm[2]  =   sympify('fac*pwo');
    idm[3]  =   sympify('fac*pfo');
    idm[4]  =   sympify('fac*pfa');
    idm[5]  =   sympify('fac*pjd');
    idm[6]  =   sympify('fac*phm');
    idm[7]  =   sympify('fac*pek');
    idm[8]  =   sympify('fac*pti');
    idm[9]  =   sympify('fac*pkj');
    idm[10] =  sympify('ph2o**2');
    

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');
    gbase[6]  = sympify('gb6');
    gbase[7]  = sympify('gb7');
    gbase[8]  = sympify('gb8');
    gbase[9]  = sympify('gb9');
    gbase[10] = sympify('gb10');


    symb     = ['wo','sl','fo','fa','jd','hm','ek','ti','kj','h2o']

    in_var  = ['wo','sl','fo','fa','jd','hm','ek','ti','kj','h2o','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','x[7]','x[8]','x[9]','x[10]','ln']
    
    in_sf = ['pq', 'psl', 'pwo', 'pfo', 'pfa', 'pjd', 'phm', 'pek', 'pti', 'pkj', 'ph2o', 'fac'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]','sf[10]','sf[11]','sf[12]','sf[13]','sf[14]','sf[15]','sf[16]','sf[17]']

    
    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def pl4T_ig():
    
    P        = symbols('P')
    T        = symbols('T')
    n_em     = 3 # 3 endmembers and 2 compositional variables

    W        = [0]*3
    v        = [0]*3
    gbase    = [0]*n_em;
    p        = [0]*n_em;
    idm      = [0]*n_em;
    stp       = 0.124;
    # Initial guess -> ca,k
    IG       = [0]*(n_em-1);
    IG[0]    = 0.8;
    IG[1]    = 0.03;
    
    # Validity bounds (need as many slack variables and Lagrange multiplier as there is range bounds)
    # Dim = n_emm - 1
    Range    = [[0]*2]*(n_em-1); # declare range
  
    Range[0] = [0,1]; # ca range
    Range[1] = [0,1]; # k range

    emname  = ['ab','an','san']
    make    = [0]*n_em;
    make[0] = sympify('ab');
    make[1] = sympify('an');
    make[2] = sympify('san');
    
    p[0]    = sympify('1 - k - ca');
    p[1]    = sympify('ca');
    p[2]    = sympify('k');

    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0]    = 14.6 - 0.00935*T - 0.04*P;
    W[1]    = 24.1 - 0.00957*T + 0.338*P;
    W[2]    = 48.5 - 0.13*P;
    v[0]    = 0.674;
    v[1]    = 0.55;
    v[2]    = 1.0;

    #     Site fractions
    xNaA  = sympify('1 - ca - k');
    xCaA  = sympify('ca');
    xKA   = sympify('k');
    xAlTB = sympify('1/4 + 1/4*ca');
    xSiTB = sympify('3/4 - 1/4*ca');

    sf = [xNaA, xCaA, xKA, xAlTB, xSiTB];
 
    #     Ideal mixing parameters
    idm[0] =   sympify('1.7548*xNaA*xAlTB**(1/4)*xSiTB**(3/4)');
    idm[1] =   sympify('2.0*xCaA*xAlTB**(1/2)*xSiTB**(1/2)');
    idm[2] =   sympify('1.7548*xKA*xAlTB**(1/4)*xSiTB**(3/4)');

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');

    symb     = ['ca','k']
    
    in_var  = ['ca','k','log']   
    out_var = ['x[0]','x[1]','ln']
    
    in_sf = ['xNaA', 'xCaA', 'xKA', 'xAlTB', 'xSiTB'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]']
    
    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def ol_ig():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 4

    W         = [0]*6
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.09;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [0,1]; # c range
    Range[2]  = [0,1]; # Q range

    emname    = ['mont','fa','fo','cfm']
    make      = [0]*n_em
    make[0]   = sympify('mont');
    make[1]   = sympify('fa');
    make[2]   = sympify('fo');
    make[3]   = sympify('1/2 *fa + 1/2 *fo');
 
    p[0] = sympify('c');
    p[1] = sympify('-q + x');
    p[2] = sympify('1 - c - q - x + c*x');
    p[3] = sympify('2*q + (-c)*x');
    
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 24.;
    W[1] = 38.;
    W[2] = 24.;
    W[3] = 9.;
    W[4] = 4.5;
    W[5] = 4.5;    
    v    = 0.

    #     Site fractions
    xMgM1 = sympify('1 + q - x');
    xFeM1 = sympify('-q + x');
    xMgM2 = sympify('1 - c - q - x + c*x');
    xFeM2 = sympify('q + x + (-c)*x');
    xCaM2 = sympify('c');
    
    idm2[0] =   xMgM1*xCaM2;
    idm2[1] =   xFeM1*xFeM2;
    idm2[2] =   xMgM1*xMgM2;
    idm2[3] =   xMgM1*xFeM2;
    
    sf    = [xMgM1, xFeM1, xMgM2, xFeM2, xCaM2];

    #     Ideal mixing parameters
    idm[0] =   sympify('xMgM1*xCaM2');
    idm[1] =   sympify('xFeM1*xFeM2');
    idm[2] =   sympify('xMgM1*xMgM2');
    idm[3] =   sympify('xMgM1*xFeM2');

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    
    symb     = ['x','c','q']

    in_var  = ['x','c','q','log']   
    out_var = ['x[0]','x[1]','x[2]','ln']

    in_sf    = ['xMgM1', 'xFeM1', 'xMgM2', 'xFeM2', 'xCaM2'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]']

    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def mu_ig():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 6

    W         = [0]*15
    v         = [0]*6
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.19;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [0,1]; # y range
    Range[2]  = [0,1]; # f range
    Range[3]  = [0,1]; # n range
    Range[4]  = [0,1]; # c range

    emname    = ['mu','cel','fcel','pa','mam','fmu']
    make      = [0]*n_em
    make[0]   = sympify('mu');
    make[1]   = sympify('cel');
    make[2]   = sympify('fcel');
    make[3]   = sympify('pa');
    make[4]   = sympify('ma + 6.5');
    make[5]   = sympify('1/2 *andr - 1/2* gr + mu + 25');
 
    p[0] = sympify('-c - f - n + y');
    p[1] = sympify('1. - x - y + x*y');
    p[2] = sympify('x + (-x)*y');
    p[3] = sympify('n');
    p[4] = sympify('c');
    p[5] = sympify('f');

    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 0. + 0.20*P;
    W[1] = 0. + 0.20*P;
    W[2] = 10.12 + 0.0034*T + 0.353*P;
    W[3] = 35.0;
    W[4] = 0.;
    W[5] = 0.;
    W[6] = 45.0 + 0.25*P;
    W[7] = 50.0;
    W[8] = 0.;
    W[9] = 45.0 + 0.25*P;
    W[10] = 50.0;
    W[11] = 0.;
    W[12] = 15.0;
    W[13] = 30.0;
    W[14] = 35.0;
    v[0] = 0.63;
    v[1] = 0.63;
    v[2] = 0.63;
    v[3] = 0.37;
    v[4] = 0.63;
    v[5] = 0.63;

    #     Site fractions
    xKA     = sympify('1. - c - n');
    xNaA    = sympify('n');
    xCaA    = sympify('c');
    xMgM2A  = sympify('1. - x - y + x*y');
    xFeM2A  = sympify('x + (-x)*y');
    xAlM2A  = sympify('y');
    xAlM2B  = sympify('1. - f');
    xFe3M2B = sympify('f');
    xSiT1   = sympify('1. - 1./2.*c - 1./2.*y');
    xAlT1   = sympify('1./2.*c + 1./2.*y'); 
    
    sf      = [xKA, xNaA, xCaA, xMgM2A, xFeM2A, xAlM2A, xAlM2B, xFe3M2B, xSiT1, xAlT1];

    #     Ideal mixing parameters
    idm[0] = sympify('4*xKA*xAlM2A*xAlM2B*xSiT1*xAlT1');
    idm[1] = sympify('xKA*xMgM2A*xAlM2B*(xSiT1**2)');
    idm[2] = sympify('xKA*xFeM2A*xAlM2B*(xSiT1**2)');
    idm[3] = sympify('4*xNaA*xAlM2A*xAlM2B*xSiT1*xAlT1');
    idm[4] = sympify('xCaA*xAlM2A*xAlM2B*(xAlT1**2)');
    idm[5] = sympify('4*xKA*xAlM2A*xFe3M2B*xSiT1*xAlT1');

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');

    symb     = ['x','y','f','n','c']

    in_var  = ['x','y','f','n','c','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','ln']
    in_sf   = ['xKA', 'xNaA', 'xCaA', 'xMgM2A', 'xFeM2A', 'xAlM2A', 'xAlM2B', 'xFe3M2B', 'xSiT1', 'xAlT1'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]']

    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def bi_ig():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 6

    W         = [0]*15
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.12;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [0,1]; # y range
    Range[2]  = [0,1]; # f range
    Range[3]  = [0,1]; # t range
    Range[4]  = [0,1]; # Q range
    
    emname  =['phl','annm','obi','east','tbi','fbi']
    make    = [0]*n_em;
    make[0] = sympify('phl');
    make[1] = sympify('ann - 6');
    make[2] = sympify('1/3 *ann + 2/3 *phl - 6');
    make[3] = sympify('east ');
    make[4] = sympify('- br + phl + ru + 55');
    make[5] = sympify('1/2 *andr + east - 1/2 *gr - 3');

    p[0] = sympify('1.0 - f - t - x - y - 2.0/3.0*q + f*x + t*x + x*y');
    p[1] = sympify('-1/3*q + x');
    p[2] = sympify('q -f*x -t*x -x*y');
    p[3] = sympify('y');
    p[4] = sympify('t');
    p[5] = sympify('f');

    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 12.;
    W[1] = 4.;
    W[2] = 10.;
    W[3] = 30.;
    W[4] = 8.;
    W[5] = 8.;
    W[6] = 5.;
    W[7] = 32.;
    W[8] = 13.6;
    W[9] = 7.;
    W[10] = 24.;
    W[11] = 5.6;
    W[12] = 40.0;
    W[13] = 1.0;
    W[14] = 40.0;
    v = 0.;

    #     Site fractions
    xMgM3 = sympify('1 - f - t - x - y - 2/3*q + f*x + t*x + x*y');
    xFeM3 = sympify('x + 2/3*q + (-f)*x + (-t)*x + (-x)*y');
    xFe3M3 = sympify('f');
    xTiM3 = sympify('t');
    xAlM3 = sympify('y');
    xMgM12 = sympify('1 + 1/3*q - x');
    xFeM12 = sympify('-1/3*q + x');
    xSiT = sympify('1/2 - 1/2*f - 1/2*y');
    xAlT = sympify('1/2 + 1/2*f + 1/2*y');
    xOHV = sympify('1 - t');
    xOV = sympify('t');
    
    sf  = [xMgM3, xFeM3, xFe3M3, xTiM3, xAlM3, xMgM12, xFeM12, xSiT, xAlT, xOHV, xOV];
     
    #     Ideal mixing parameters
    idm[0] = sympify('4*xMgM3*xMgM12**2*xSiT*xAlT*xOHV**2');
    idm[1] = sympify('  4*xFeM3*xFeM12**2*xSiT*xAlT*xOHV**2');
    idm[2] = sympify('  4*xFeM3*xMgM12**2*xSiT*xAlT*xOHV**2');
    idm[3] = sympify('xAlM3*xMgM12**2*xAlT**2*xOHV**2');
    idm[4] = sympify('  4*xTiM3*xMgM12**2*xSiT*xAlT*xOV**2');
    idm[5] = sympify('xFe3M3*xMgM12**2*xAlT**2*xOHV**2');

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');

    
    symb     = ['x','y','f','t','q']

    in_var  = ['x','y','f','t','q','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','ln']
    
    in_sf  = ['xMgM3', 'xFeM3', 'xFe3M3', 'xTiM3', 'xAlM3', 'xMgM12', 'xFeM12', 'xSiT', 'xAlT', 'xOHV', 'xOV']; 
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]','sf[10]','sf[11]']
    
    
    
    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def g_ig():
    P = symbols('P')
    T = symbols('T')
    n_em = 6

    W         = [0]*15
    v         = [0]*6
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.19;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x  range
    Range[1]  = [0,1]; # c  range
    Range[2]  = [0,1]; # f  range
    Range[3]  = [0,1]; # cr range
    Range[4]  = [0,1]; # t  range


    emname    = ['py','alm','gr','andr','knom','tig']
    make      = [0]*n_em
    make[0]   = sympify('py');
    make[1]   = sympify('alm');
    make[2]   = sympify('gr');
    make[3]   = sympify('andr');
    make[4]   = sympify('knor + 18.2');
    make[5]   = sympify('py + 1/2 *e_per + 1/2 *ru - 1/2 *cor + 46.7 - 0.0173 *T');
 
    
    p[0] = sympify('1 - c - cr - x - 4*t + c*x');
    p[1] = sympify('x + (-c)*x');
    p[2] = sympify('c - f');
    p[3] = sympify('f');
    p[4] = sympify('cr');
    p[5] = sympify('4*t');

    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 4.0 + 0.10*P;
    W[1] = 45.4 - 0.010*T + 0.04*P;
    W[2] = 107.0 - 0.010*T - 0.036*P;
    W[3] = 2.0;
    W[4] = 0;
    W[5] = 17.0 - 0.010*T + 0.10*P;
    W[6] = 65.0 - 0.010*T + 0.039*P;
    W[7] = 6.0 + 0.01*P;
    W[8] = 0;
    W[9] = 2.0;
    W[10] = 1.0 - 0.010*T + 0.18*P;
    W[11] = 0;
    W[12] = 63.0 - 0.010*T + 0.10*P;
    W[13] = 0;
    W[14] = 0;
    v[0] = 1.00;
    v[1] = 1.00;
    v[2] = 2.50;
    v[3] = 2.50;
    v[4] = 1.00;
    v[5] = 1.00;

    #     Site fractions
    xMgM1  = sympify('1 - c - x + c*x');
    xFeM1  = sympify('x + (-c)*x');
    xCaM1  = sympify('c');
    xAlM2  = sympify('1 - cr - f - 2*t');
    xCrM2  = sympify('cr');
    xFe3M2 = sympify('f');
    xMgM2  = sympify('t');
    xTiM2  = sympify('t');

    sf    = [xMgM1, xFeM1, xCaM1, xAlM2, xCrM2, xFe3M2, xMgM2, xTiM2];      
    
    #     Ideal mixing parameters
    idm[0] =   sympify('xMgM1**3*xAlM2**2');
    idm[1] =   sympify('xFeM1**3*xAlM2**2');
    idm[2] =   sympify('xCaM1**3*xAlM2**2');
    idm[3] =   sympify('xCaM1**3*xFe3M2**2');
    idm[4] =   sympify('xMgM1**3*xCrM2**2');
    idm[5] = sympify('8*xMgM1**3*xAlM2*xMgM2**(1/2)*xTiM2**(1/2)');

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');

    symb     = ['x','c','f','cr','t']
    
    in_var  = ['x','cr','f','c','t','log']   
    out_var = ['x[0]','x[3]','x[2]','x[1]','x[4]','ln']
    
    in_sf  = ['xMgM1', 'xFeM1', 'xCaM1', 'xAlM2', 'xCrM2', 'xFe3M2', 'xMgM2', 'xTiM2']; 
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]','sf[10]','sf[11]']
    
    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def ep_ig():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 3

    W         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.049;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];   # f range
    Range[1]  = [0,0.5]; # Q range

    emname    = ['cz','ep','fep']
    make      = [0]*n_em
    make[0]   = sympify('cz');
    make[1]   = sympify('ep');
    make[2]   = sympify('fep');

    p[0] = sympify('1 - f - q');
    p[1] = sympify('2*q');
    p[2] = sympify('f - q');
    
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 1.;
    W[1] = 3.;
    W[2] = 1.;    
    v = 0.; 

    #     Site fractions
    xFeM1 = sympify('f - q');
    xAlM1 = sympify('1 - f + q');
    xFeM3 = sympify('f + q');
    xAlM3 = sympify('1 - f - q');
 
    sf    = [xFeM1, xAlM1, xFeM3, xAlM3];

    #     Ideal mixing parameters
    idm[0] =   sympify('xAlM1*xAlM3');
    idm[1] =   sympify('xAlM1*xFeM3');
    idm[2] =   sympify('xFeM1*xFeM3');
    
    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');

    symb     = ['f','q']
    
    in_var  = ['f','q','log']   
    out_var = ['x[0]','x[1]','ln']
    in_sf    = ['xFeM1', 'xAlM1', 'xFeM3', 'xAlM3'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]']

    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def cd_ig():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 3

    W         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.09;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [0,1]; # h range

    emname    = ['crd','fcrd','hcrd']
    make      = [0]*n_em
    make[0]   = sympify('crd');
    make[1]   = sympify('fcrd');
    make[2]   = sympify('hcrd');

    p[0] = sympify('1 - h - x');
    p[1] = sympify('x');
    p[2] = sympify('h');
    
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 6.;
    W[1] = 0.;
    W[2] = 0.;    
    v = 0.; 

    #     Site fractions
    xFeX  = sympify('x');
    xMgX  = sympify('1 - x');
    xH2OH = sympify('h');
    xvH   = sympify('1 - h');

    sf  = [xFeX, xMgX, xH2OH, xvH];

    #     Ideal mixing parameters
    idm[0] =   sympify('xMgX**2*xvH');
    idm[1] =   sympify('xFeX**2*xvH');
    idm[2] =   sympify('xMgX**2*xH2OH');
    
    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');

    symb     = ['x','h']

    in_var  = ['x','h','log']   
    out_var = ['x[0]','x[1]','ln']
    in_sf  = ['xFeX', 'xMgX', 'xH2OH', 'xvH'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]']

    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def opx_ig():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 9
    idm2 = []

    W         = [0]*36
    v         = [0]*9
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.249;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];  # x range
    Range[1]  = [0,2];  # y range
    Range[2]  = [0,1];  # c range         # Range(3,:) = [ 0 0.5], Boris correction commented
    Range[3]  = [-1,1]; # Q range
    Range[4]  = [0,1];  # f range
    Range[5]  = [0,1];  # t range
    Range[6]  = [0,1];  # cr range
    Range[7]  = [0,1];  # j range
    

    emname    = ['en','fs','fm','odi','mgts','cren','obuf','mess','ojd']
    make      = [0]*n_em
    make[0]   = sympify('en');
    make[1]   = sympify('fs');
    make[2]   = sympify('1/2 *en + 1/2 *fs - 6.6');
    make[3]   = sympify('di + 2.8 + 0.005* P');
    make[4]   = sympify('mgts');
    make[5]   = sympify('mgts + kos - jd - 25.9 + 0.0155* T + 0.05 *P');
    make[6]   = sympify('mgts + 1/2 *e_per + 1/2 *ru - 1/2 *cor - 5 - 0.0051 *T - 0.0061* P');
    make[7]   = sympify('mgts + acm - jd + 4.8 - 0.089 *P');
    make[8]   = sympify('jd + 18.8');

    
    p[0] = sympify('1 - c - j + q - x - y + (-j)*q + q*t + c*x + j*x + (-q)*y');
    p[1] = sympify('q + x + (-j)*q + q*t + (-j)*x + t*x + (-q)*y + (-x)*y');
    p[2] = sympify('-2*q + 2*j*q + (-2*q)*t + (-c)*x + (-t)*x + 2*q*y + x*y');
    p[3] = sympify('c');
    p[4] = sympify('-cr - f + y - 2*t');
    p[5] = sympify('cr');
    p[6] = sympify('2*t');
    p[7] = sympify('f');
    p[8] = sympify('j');

    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 7.0;
    W[1] = 4.0;
    W[2] = 29.4;
    W[3] = 12.5 - 0.04*P;
    W[4] = 8.0;
    W[5] = 6.0;
    W[6] = 8.0;
    W[7] = 35.0;
    W[8] = 4.0;
    W[9] = 21.5 + 0.08*P;
    W[10] = 11.0 - 0.15*P;
    W[11] = 10.0;
    W[12] = 7.0;
    W[13] = 10.0;
    W[14] = 35.0;
    W[15] = 18.0 + 0.08*P;
    W[16] = 15.0 - 0.15*P;
    W[17] = 12.0;
    W[18] = 8.0;
    W[19] = 12.0;
    W[20] = 35.0;
    W[21] = 75.5 - 0.84*P;
    W[22] = 20.0;
    W[23] = 40.0;
    W[24] = 20.0;
    W[25] = 35.0;
    W[26] = 2.0;
    W[27] = 10.0;
    W[28] = 2.0;
    W[29] = 7.0;
    W[30] = 6.0;
    W[31] = 2.0;
    W[32] = -11.0;
    W[33] = 6.0;
    W[34] = 20.0;
    W[35] = -11.0;
    v[0] = 1.00;
    v[1] = 1.00;
    v[2] = 1.00;
    v[3] = 1.20;
    v[4] = 1.00;
    v[5] = 1.00;
    v[6] = 1.00;
    v[7] = 1.00;
    v[8] = 1.20;

    #     Site fractions
    xMgM1  = sympify('1 - j - q + t - x - y + j*q + (-q)*t + j*x + (-t)*x + q*y + x*y');
    xFeM1  = sympify('q + x + (-j)*q + q*t + (-j)*x + t*x + (-q)*y + (-x)*y');
    xAlM1  = sympify('-cr - f + j + y - 2*t');
    xFe3M1 = sympify('f');
    xCrM1  = sympify('cr');
    xTiM1  = sympify('t');
    xMgM2  = sympify('1 - c - j + q - x + (-j)*q + q*t + c*x + j*x + (-q)*y');
    xFeM2  = sympify('-q + x + j*q + (-q)*t + (-c)*x + (-j)*x + q*y');
    xCaM2  = sympify('c');
    xNaM2  = sympify('j');
    xSiT   = sympify('1 - 1/2*y');
    xAlT   = sympify('1/2*y');

    sf   = [xMgM1, xFeM1, xAlM1, xFe3M1, xCrM1, xTiM1, xMgM2, xFeM2, xCaM2, xNaM2, xSiT, xAlT];

    #     Ideal mixing parameters
    idm[0] =   sympify('xMgM1*xMgM2*xSiT**(1/2)');
    idm[1] =   sympify('xFeM1*xFeM2*xSiT**(1/2)');
    idm[2] =   sympify('xMgM1*xFeM2*xSiT**(1/2)');
    idm[3] =   sympify('xMgM1*xCaM2*xSiT**(1/2)');
    idm[4] = sympify('1.4142*xAlM1*xMgM2*xSiT**(1/4)*xAlT**(1/4)');
    idm[5] = sympify('1.4142*xCrM1*xMgM2*xSiT**(1/4)*xAlT**(1/4)');
    idm[6] = sympify('2.8284*xMgM1**(1/2)*xTiM1**(1/2)*xMgM2*xSiT**(1/4)*xAlT**(1/4)');
    idm[7] = sympify('1.4142*xFe3M1*xMgM2*xSiT**(1/4)*xAlT**(1/4)');
    idm[8] =   sympify('xAlM1*xNaM2*xSiT**(1/2)');

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');
    gbase[6]  = sympify('gb6');
    gbase[7]  = sympify('gb7');
    gbase[8]  = sympify('gb8');
    
    in_var  = ['x','y','cr','q','f','t','c','j','log']   
    out_var = ['x[0]','x[1]','x[6]','x[3]','x[4]','x[5]','x[2]','x[7]','ln']
    
    symb     = ['x','y','c','q','f','t','cr','j']
    in_sf   = ['xMgM1', 'xFeM1', 'xAlM1', 'xFe3M1', 'xCrM1', 'xTiM1', 'xMgM2', 'xFeM2', 'xCaM2', 'xNaM2', 'xSiT', 'xAlT'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]','sf[10]','sf[11]']

    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def cpx_ig():

    P = symbols('P')
    T = symbols('T')
    n_em = 10

    W         = [0]*45
    v         = [0]*10
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.249;
# Validity bounds
    Range    = [[0]*2]*(n_em); # declare range
    
    Range[0]  = [0,1];  # x range
    Range[1]  = [0,2];  # y range
    Range[2]  = [0,1];  # o range
    Range[3]  = [0,1]; # n range
    Range[4]  = [-1,1];  # Q range
    Range[5]  = [0,1];  # f range
    Range[6]  = [0,1];  # cr range
    Range[7]  = [0,1];  # t range
    Range[8]  = [0,1];  # k range
    emname  = ['di','cfs','cats','crdi','cess','cbuf','jd','cen','cfm','kjd']
    make    = [0]*n_em;
    make[0] = sympify('di');
    make[1] = sympify('fs + 2.1 - 0.002 *T + 0.045 *P');
    make[2] = sympify('cats');
    make[3] = sympify('d_cats + kos - jd - 4.9 ');
    make[4] = sympify('d_cats + acm - jd - 3.45');
    make[5] = sympify('d_cats + 1/2 *e_per + 1/2* ru - 1/2* cor - 16.2 - 0.0012* T - 0.005 *P');
    make[6] = sympify('jd');
    make[7] = sympify('en + 3.5 - 0.002 *T + 0.048 *P ');
    make[8] = sympify('1/2 *en + 1/2 *fs - 1.6 - 0.002 *T + 0.0465* P');
    make[9] = sympify('jd - abh + san + 11.7 + 0.6* P '); 
    
    p[0] = sympify('1 - k - n - o - y');
    p[1] = sympify('q + x + (-k)*q + (-n)*q + q*t + (-k)*x + (-n)*x + t*x + (-q)*y + (-x)*y');
    p[2] = sympify('-cr - f + y - 2*t');
    p[3] = sympify('cr');
    p[4] = sympify('f');
    p[5] = sympify('2*t');
    p[6] = sympify('n');
    p[7] = sympify('o + q + (-k)*q + (-n)*q + q*t + (-o)*x + (-q)*y');
    p[8] = sympify('-x - 2*q + 2*k*q + 2*n*q + (-2*q)*t + k*x + n*x + o*x + (-t)*x + 2*q*y + x*y');
    p[9] = sympify('k');
    
    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 25.8;
    W[1] = 13.0 - 0.06*P;
    W[2] = 8.0;
    W[3] = 8.0;
    W[4] = 8.0;
    W[5] = 26.0;
    W[6] = 29.8;
    W[7] = 20.6;
    W[8] = 26.0;
    
    W[9] = 25.0 - 0.10*P;
    W[10] = 38.3;
    W[11] = 43.3;
    W[12] = 24.0;
    W[13] = 24.0;
    W[14] = 2.3;
    W[15] = 3.5;
    W[16] = 24.0;
    
    W[17] = 2.0;
    W[18] = 2.0;
    W[19] = 6.0;
    W[20] = 6.0;
    W[21] = 45.2 - 0.35*P;
    W[22] = 27.0 - 0.10*P;
    W[23] = 6.0;
    
    W[24] = 2.0;
    W[25] = 6.0;
    W[26] = 3.0;
    W[27] = 52.3;
    W[28] = 40.3;
    W[29] = 3.0;
    
    W[30] = 6.0;
    W[31] = 3.0;
    W[32] = 57.3;
    W[33] = 45.3;
    W[34] = 3.0;
    
    W[35] = 16.0;
    W[36] = 24.0;
    W[37] = 22.0;
    W[38] = 16.0;
    
    W[39] = 40.0;
    W[40] = 40.0;
    W[41] = 28.0;
    
    W[42] = 4.0;
    W[43] = 40.0;
    W[44] = 40.0;
    v[0] = 1.20;
    v[1] = 1.00;
    v[2] = 1.90;
    v[3] = 1.90;
    v[4] = 1.90;
    v[5] = 1.90;
    v[6] = 1.20;
    v[7] = 1.00;
    v[8] = 1.00;
    v[9] = 1.20;

    #     Site fractions
    xMgM1  = sympify('1 - k - n - q + t - x - y + k*q + n*q + (-q)*t + k*x + n*x + (-t)*x + q*y + x*y');
    xFeM1  = sympify('q + x + (-k)*q + (-n)*q + q*t + (-k)*x + (-n)*x + t*x + (-q)*y + (-x)*y');
    xAlM1  = sympify('-cr - f + k + n + y - 2*t');
    xFe3M1 = sympify('f');
    xCrM1  = sympify('cr');
    xTiM1  = sympify('t');
    xMgM2  = sympify('o + q + (-k)*q + (-n)*q + q*t + (-o)*x + (-q)*y');
    xFeM2  = sympify('-q + k*q + n*q + (-q)*t + o*x + q*y');
    xCaM2  = sympify('1 - k - n - o');
    xNaM2  = sympify('n');
    xKM2   = sympify('k');
    xSiT   = sympify('1 - 1/2*y');
    xAlT   = sympify('1/2*y');

    sf   = [xMgM1, xFeM1, xAlM1, xFe3M1, xCrM1, xTiM1, xMgM2, xFeM2, xCaM2, xNaM2, xKM2, xSiT, xAlT];
    
    #     Ideal mixing parameters
    idm[0] = sympify('xMgM1*xCaM2*xSiT**(1/2)');
    idm[1] = sympify('xFeM1*xFeM2*xSiT**(1/2)');
    idm[2] = sympify('1.4142*xAlM1*xCaM2*xSiT**(1/4)*xAlT**(1/4)');
    idm[3] = sympify('1.4142*xCrM1*xCaM2*xSiT**(1/4)*xAlT**(1/4)');
    idm[4] = sympify('1.4142*xFe3M1*xCaM2*xSiT**(1/4)*xAlT**(1/4)');
    idm[5] = sympify('2.8284*xMgM1**(1/2)*xTiM1**(1/2)*xCaM2*xSiT**(1/4)*xAlT**(1/4)');
    idm[6] = sympify('xAlM1*xNaM2*xSiT**(1/2)');
    idm[7] = sympify('xMgM1*xMgM2*xSiT**(1/2)');
    idm[8] = sympify('xMgM1*xFeM2*xSiT**(1/2)');
    idm[9] = sympify('xAlM1*xKM2*xSiT**(1/2)');
 
    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');
    gbase[6]  = sympify('gb6');
    gbase[7]  = sympify('gb7');
    gbase[8]  = sympify('gb8');
    gbase[9]  = sympify('gb9');

    symb     = ['x','y','o','n','q','f','cr','t','k']

    in_var  = ['x','y','o','n','q','f','cr','t','k','log']  
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','x[7]','x[8]','ln']
    in_sf   = ['xMgM1', 'xFeM1', 'xAlM1', 'xFe3M1', 'xCrM1', 'xTiM1', 'xMgM2', 'xFeM2', 'xCaM2', 'xNaM2', 'xKM2', 'xSiT', 'xAlT'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]','sf[10]','sf[11]','sf[12]']

    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def spn_ig():
    
    P = symbols('P')
    T = symbols('T')
    
    n_em = 8

    W         = [0]*28
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.249;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];  # x  range
    Range[1]  = [0,1.]; # y  range
    Range[2]  = [0,1];  # c  range         # Range(3,:) = [ 0 0.5], Boris correction commented
    Range[3]  = [0,1]; # t  range
    Range[4]  = [-1,1];  # Q1 range
    Range[5]  = [-1,1];  # Q2 range
    Range[6]  = [-1,1];  # Q3 range

    emname  = ['nsp','isp','nhc','ihc','nmt','imt','pcr','qndm']
    make    = [0]*n_em;
    make[0] = sympify('o_sp');
    make[1] = sympify('o_sp + 23.6 - 0.005763 *T');
    make[2] = sympify('o_herc');
    make[3] = sympify('o_herc + 23.6 - 0.005763 *T');
    make[4] = sympify('e_mt + 0 + 0.005763* T');
    make[5] = sympify('e_mt + 0.3');
    make[6] = sympify('picr');
    make[7] = sympify('qnd - 30');

    
    p[0] = sympify('1/3 - 1/3*x - c + 2/3*Q1 - 2/3*t + (-1/3*t)*x');
    p[1] = sympify('2/3 - 1/3*t - 2/3*Q1 - 2/3*x + (-2/3*t)*x');
    p[2] = sympify('1/3*x - 1/3*y + 2/3*Q2 + 2/3*Q3 + 1/3*t*x + 1/3*c*y + 1/3*t*y');
    p[3] = sympify('-2/3*Q2 - 2/3*Q3 + 2/3*x - 2/3*y + 2/3*t*x + 2/3*c*y + 2/3*t*y');
    p[4] = sympify('1/3*y - 2/3*Q3 + (-1/3*c)*y + (-1/3*t)*y');
    p[5] = sympify('2/3*Q3 + 2/3*y + (-2/3*c)*y + (-2/3*t)*y');
    p[6] = sympify('c');
    p[7] = sympify('t');

    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = -8.2;
    W[1] = 3.5;
    W[2] = -13.0;
    W[3] = 43.2;
    W[4] = 49.1;
    W[5] = -5.0;
    W[6] = 22.5;
    W[7] = 4.4;
    W[8] = -6.0;
    W[9] = 36.8;
    W[10] = 20.0;
    W[11] = 14.0;
    W[12] = 21.5;
    W[13] = -8.2;
    W[14] = 18.1;
    W[15] = 49.0;
    W[16] = -19.0;
    W[17] = 35.1;
    W[18] = -4.0;
    W[19] = 7.6;
    W[20] = -11.0;
    W[21] = 9.0;
    W[22] = 18.1;
    W[23] = 11.9;
    W[24] = 62.2;
    W[25] = -6.4;
    W[26] = 24.3;
    W[27] = 60.0;
    v = 0.;

    #     Site fractions
    xMgT  = sympify('1/3 + 1/3*t - 1/3*x + 2/3*Q1 + (-1/3*t)*x');
    xFeT  = sympify('1/3*x + 2/3*Q2 + 1/3*t*x');
    xAlT  = sympify('2/3 - 1/3*t - 2/3*Q1 - 2/3*Q2 - 2/3*Q3 - 2/3*y + 2/3*c*y + 2/3*t*y');
    xFe3T = sympify('2/3*Q3 + 2/3*y + (-2/3*c)*y + (-2/3*t)*y');
    xMgM  = sympify('1/3 - 1/3*Q1 + 1/3*t - 1/3*x + (-1/3*t)*x');
    xFeM  = sympify('-1/3*Q2 + 1/3*x + 1/3*t*x');
    xAlM  = sympify('2/3 + 1/3*Q1 + 1/3*Q2 + 1/3*Q3 - c - 2/3*y - 5/6*t + 2/3*c*y + 2/3*t*y');
    xFe3M = sympify('-1/3*Q3 + 2/3*y + (-2/3*c)*y + (-2/3*t)*y');
    xCrM  = sympify('c');
    xTiM  = sympify('1/2*t');

    sf   = [xMgT, xFeT, xAlT, xFe3T, xMgM, xFeM, xAlM, xFe3M, xCrM, xTiM];

    idm[0] = sympify('  xMgT*xAlM');
    idm[1] = sympify('2*xAlT*xMgM**(.5)*xAlM**(.5)');
    idm[2] = sympify('  xFeT*xAlM');
    idm[3] = sympify('2*xAlT*xFeM**(.5)*xAlM**(.5)');
    idm[4] = sympify('  xFeT*xFe3M');
    idm[5] = sympify('2*xFe3T*xFeM**(.5)*xFe3M**(.5)');
    idm[6] = sympify('  xMgT*xCrM');
    idm[7] = sympify('2*xMgT*xMgM**(.5)*xTiM**(.5)');
      
    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');
    gbase[6]  = sympify('gb6');
    gbase[7]  = sympify('gb7');

    symb     = ['x','y','c','t','Q1','Q2','Q3']

    in_var  = ['x','y','c','t','Q1','Q2','Q3','log']  
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','ln']
    in_sf   = ['xMgT', 'xFeT', 'xAlT', 'xFe3T', 'xMgM', 'xFeM', 'xAlM', 'xFe3M', 'xCrM', 'xTiM'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]']
 
    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def hb_ig():
    
    P = symbols('P')
    T = symbols('T')
    eps_hb = symbols('eps')
    n_em = 11

    W         = [0]*55
    v         = [0]*11
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.249;
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];  # x range
    Range[1]  = [0,2];  # y range
    Range[2]  = [0,1];  # z range         # Range(3,:) = [ 0 0.5], Boris correction commented
    Range[3]  = [0,1];  # a range
    Range[4]  = [0,1];  # k range
    Range[5]  = [0,1];  # c range
    Range[6]  = [0,1];  # f range
    Range[7]  = [0,1];  # t range
    Range[8]  = [-1,1]; # Q1 range
    Range[9]  = [-1,1]; # Q2 range

    emname  = ['tr','tsm','prgm','glm','cumm','grnm','a','b','mrb','kprg','tts']
    make    = [0]*n_em;
    make[0] = sympify('tr');
    make[1] = sympify('ts+10');
    make[2] = sympify('parg - 10');
    make[3] = sympify('gl - 3');
    make[4] = sympify('cumm');
    make[5] = sympify('grun - 3');
    make[6] = sympify('3/7 *cumm + 4/7 *grun - 11.2');
    make[7] = sympify('2/7 *cumm + 5/7 *grun - 13.8');
    make[8] = sympify('gl - gr + andr');
    make[9] = sympify('mu - pa + parg - 7.06 + 0.02 *T');
    make[10] = sympify('- 2 *dsp + 2 *ru + ts + 95');

    
    p[0] = sympify('-1/2*a + c - f - t - y + z');
    p[1] = sympify('-1/2*a + f + y - z');
    p[2] = sympify('a + (-a)*k');
    p[3] = sympify('-f + z');
    p[4] = sympify('1 - c - Q2 - x - z - 3/2*Q1 + f*Q2 + Q2*t + c*x + Q2*y + x*z');
    p[5] = sympify('x - 2*Q2 - 5/2*Q1 + 2*f*Q2 + 2*Q2*t + c*x + (-f)*x + (-t)*x + 2*Q2*y + (-x)*y + x*z');
    p[6] = sympify('Q2 + 5/2*Q1 + (-f)*Q2 + (-Q2)*t + (-c)*x + (-Q2)*y + (-x)*z');
    p[7] = sympify('2*Q2 + 3/2*Q1 + (-2*f)*Q2 + (-2*Q2)*t + (-c)*x + f*x + t*x + (-2*Q2)*y + x*y + (-x)*z');
    p[8] = sympify('f');
    p[9] = sympify('a*k');
    p[10] = sympify('t'); 

    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 20.0;
    W[1] = 25.0;
    W[2] = 65.0;
    W[3] = 45.0;
    W[4] = 75.0;
    W[5] = 57.0;
    W[6] = 63.0;
    W[7] = 52.0;
    W[8] = 30.0;
    W[9] = 85.0;
    W[10] = -40.0;
    W[11] = 25.0;
    W[12] = 70.0;
    W[13] = 80.0;
    W[14] = 70.0;
    W[15] = 72.5;
    W[16] = 20.0;
    W[17] = -40.0;
    W[18] = 35.0;
    W[19] = 50.0;
    W[20] = 90.0;
    W[21] = 106.7;
    W[22] = 94.8;
    W[23] = 94.8;
    W[24] = 40.0;
    W[25] = 8.0;
    W[26] = 15.0;
    W[27] = 100.0;
    W[28] = 113.5;
    W[29] = 100.0;
    W[30] = 111.2;
    W[31] = 0.0;
    W[32] = 54.0;
    W[33] = 75.0;
    W[34] = 33.0;
    W[35] = 18.0;
    W[36] = 23.0;
    W[37] = 80.0;
    W[38] = 87.0;
    W[39] = 100.0;
    W[40] = 12.0;
    W[41] = 8.0;
    W[42] = 91.0;
    W[43] = 96.0;
    W[44] = 65.0;
    W[45] = 20.0;
    W[46] = 80.0;
    W[47] = 94.0;
    W[48] = 95.0;
    W[49] = 90.0;
    W[50] = 94.0;
    W[51] = 95.0;
    W[52] = 50.0;
    W[53] = 50.0;
    W[54] = 35.0;
    v[0] = 1.00;
    v[1] = 1.50;
    v[2] = 1.70;
    v[3] = 0.80;
    v[4] = 1.00;
    v[5] = 1.00;
    v[6] = 1.00;
    v[7] = 1.00;
    v[8] = 0.80;
    v[9] = 1.70;
    v[10] = 1.50;

    #     Site fractions
    xvA     = sympify('1 - a');
    xNaA    = sympify('a + (-a)*k');
    xKA     = sympify('a*k');
    xMgM13  = sympify('1 + Q1 - x');
    xFeM13  = sympify('-Q1 + x');
    xMgM2   = sympify('1 - f + Q2 - t - x - y + (-f)*Q2 + (-Q2)*t + f*x + t*x + (-Q2)*y + x*y');
    xFeM2   = sympify('-Q2 + x + f*Q2 + Q2*t + (-f)*x + (-t)*x + Q2*y + (-x)*y');
    xAlM2   = sympify('y');
    xFe3M2  = sympify('f');
    xTiM2   = sympify('t');
    xCaM4   = sympify('c');
    xMgM4   = sympify('1 - c - Q2 - x - z - 3/2*Q1 + f*Q2 + Q2 *t + c*x + Q2*y + x*z');
    xFeM4   = sympify('Q2 + x + 3/2*Q1 + (-f)*Q2 + (-Q2)*t + (-c)*x + (-Q2)*y + (-x)*z');
    xNaM4   = sympify('z');
    xSiT1   = sympify('1 - 1/2*f - 1/2*t - 1/2*y + 1/2*z - 1/4*a');
    xAlT1   = sympify('1/2*f + 1/2*t + 1/2*y - 1/2*z + 1/4*a');
    xOHV    = sympify('1 - t');
    xOV     = sympify('t');
    
    sf  = [xvA, xNaA, xKA, xMgM13, xFeM13, xMgM2, xFeM2, xAlM2, xFe3M2, xTiM2, xCaM4, xMgM4, xFeM4, xNaM4, xSiT1, xAlT1, xOHV, xOV];
    #     Ideal mixing parameters
    idm[0] =   sympify('xvA*xMgM13**3*xMgM2**2*xCaM4**2*xSiT1*xOHV**2');
    idm[1] = sympify('2*xvA*xMgM13**3*xAlM2**2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOHV**2');
    idm[2] = sympify('8*xNaA*xMgM13**3*xMgM2*xAlM2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOHV**2');
    idm[3] =   sympify('xvA*xMgM13**3*xAlM2**2*xNaM4**2*xSiT1*xOHV**2');
    idm[4] =   sympify('xvA*xMgM13**3*xMgM2**2*xMgM4**2*xSiT1*xOHV**2');
    idm[5] =   sympify('xvA*xFeM13**3*xFeM2**2*xFeM4**2*xSiT1*xOHV**2');
    idm[6] =   sympify('xvA*xMgM13**3*xFeM2**2*xFeM4**2*xSiT1*xOHV**2');
    idm[7] =   sympify('xvA*xFeM13**3*xMgM2**2*xFeM4**2*xSiT1*xOHV**2');
    idm[8] =   sympify('xvA*xMgM13**3*xFe3M2**2*xNaM4**2*xSiT1*xOHV**2');
    idm[9] = sympify('8*xKA*xMgM13**3*xMgM2*xAlM2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOHV**2');
    idm[10] = sympify('2*xvA*xMgM13**3*xTiM2**2*xCaM4**2*xSiT1**(1/2)*xAlT1**(1/2)*xOV**2');

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');
    gbase[6]  = sympify('gb6');
    gbase[7]  = sympify('gb7');
    gbase[8]  = sympify('gb8');
    gbase[9]  = sympify('gb9');
    gbase[10] = sympify('gb10');
   
    symb     = ['x','y','z','a','k','c','f','t','Q1','Q2']

    in_var  = ['x','y','z','a','k','c','f','t','Q1','Q2','log']  
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','x[7]','x[8]','x[9]','ln']
    in_sf  = ['xvA', 'xNaA', 'xKA', 'xMgM13', 'xFeM13', 'xMgM2', 'xFeM2', 'xAlM2', 'xFe3M2', 'xTiM2', 'xCaM4', 'xMgM4', 'xFeM4', 'xNaM4', 'xSiT1', 'xAlT1', 'xOHV', 'xOV'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]','sf[10]','sf[11]','sf[12]','sf[13]','sf[14]','sf[15]','sf[16]','sf[17]']
 
    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def ilm_ig():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 3

    W         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.049;
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [-0.99,0.99]; # Q range
  
    make    = [0]*n_em;
    make[0] = sympify('d_ilm');
    make[1] = sympify('d_ilm');
    make[2] = sympify('d_hem');


    p[0] = sympify('q');
    p[1] = sympify('x - q');
    p[2] = sympify('1 - x');
    
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 15.6;
    W[1] = 26.6;
    W[2] = 11.0;    
    v = 0.0; 

    #     Site fractions
    xFe2A = sympify('1/2*x + 1/2*q');
    xTiA  = sympify('1/2*x - 1/2*q');
    xFe3A = sympify('1 - x');
    xFe2B = sympify('1/2*x - 1/2*q');
    xTiB  = sympify('1/2*x + 1/2*q');
    xFe3B = sympify('1 - x');

    sf    = [xFe2A, xTiA, xFe3A, xFe2B, xTiB, xFe3B];

    #     Ideal mixing parameters
    idm[0] =   sympify('xFe2A*xTiB');
    idm[1] =   sympify('4.*xFe2A**(0.5)*xTiA**(0.5)*xFe2B**(0.5)*xTiB**(0.5)');
    idm[2] =   sympify('xFe3A*xFe3B');

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');

    symb     = ['x','q']

    in_var  = ['x','q','log']  
    out_var = ['x[0]','x[1]','ln']
    in_sf   = ['xFe2A', 'xTiA', 'xFe3A', 'xFe2B', 'xTiB', 'xFe3B']; 
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]']

    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make

def ilm_TJBH_2019_ig():
    
    P = symbols('P')
    T = symbols('T')
    n_em = 3

    W         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.049;
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1]; # x range
    Range[1]  = [-0.99,0.99]; # Q range
    
    emname  = ['oilm','dilm','dhem']
    make    = [0]*n_em;
    make[0] = sympify('o_ilm');
    make[1] = sympify('d_ilm + 1.9928 - 0.0021 *T');
    make[2] = sympify('e_hem');

    p[0] = sympify('q');
    p[1] = sympify('i - q');
    p[2] = sympify('1 - i');
    
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 7.05;
    W[1] = 14.3;
    W[2] = 7.25;    
    v = 0.; 

    #     Site fractions
    xFeA  = sympify('1/2*i + 1/2*q');
    xTiA  = sympify('1/2*i - 1/2*q');
    xFe3A = sympify('1 - i');
    xFeB  = sympify('1/2*i - 1/2*q');
    xTiB  = sympify('1/2*i + 1/2*q');
    xFe3B = sympify('1 - i');

    sf    = [xFeA, xTiA, xFe3A, xFeB, xTiB, xFe3B];

    #     Ideal mixing parameters
    idm[0] =   sympify('xFeA**(0.5)*xTiB**(0.5)');
    idm[1] =   sympify('2.*xFeA**(0.25)*xTiA**(0.25)*xFeB**(0.25)*xTiB**(0.25)');
    idm[2] =   sympify('xFe3A**(0.5)*xFe3B**(0.5)');

    #     Gbase must be initiated as define below, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');

    symb     = ['i','q']

    in_var  = ['i','q','log']  
    out_var = ['x[0]','x[1]','ln']
    in_sf    = ['xFeA', 'xTiA', 'xFe3A', 'xFeB', 'xTiB', 'xFe3B'];
    out_sf = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]']


    return symb, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname


In [110]:
# ss database metapelite _mp n=16

def liq_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 8

    W         = [0]*28
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.249;
    IG        = [0]*(n_em-1);
    cv        = [0]*(n_em-1);
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1]; 
    Range[3]  = [0,1];
    Range[4]  = [0,1];
    Range[5]  = [0,1];
    Range[6]  = [0,1];

    emname    = ['q4L','abL','kspL','anL','slL','fo2L','fa2L','h2oL']
    make      = [0]*n_em
    make[0]   = sympify('4.0 *qL')
    make[1]   = sympify('abL')
    make[2]   = sympify('kspL')
    make[3]   = sympify('anL')
    make[4]   = sympify('8/5* silL - 23')
    make[5]   = sympify('2 *foL - 10')
    make[6]   = sympify('2 *faL - 9 - 1.3* P')
    make[7]   = sympify('h2oL')

    
    p[0] = sympify('q');
    p[1] = sympify('fsp*na');
    p[2] = sympify('fsp*(1 - na)');
    p[3] = sympify('an');
    p[4] = sympify('1 - q - fsp - an - ol - h2o');
    p[5] = sympify('ol*(1 - xc)');
    p[6] = sympify('ol*xc');
    p[7] = sympify('h2o');
    
    #     Site fractions
    fac  = sympify('1 - h2o');
    pq   = sympify('q');
    xab  = sympify('fsp*na');
    xksp = sympify('fsp*(1 - na)');
    pan  = sympify('an');
    psil = sympify('1 - q - fsp - an - ol - h2o');
    pol  = sympify('ol');
    xFe  = sympify('xc');
    xMg  = sympify('1 - xc');
    ph2o = sympify('h2o');

    sf = [fac,pq,xab,xksp,pan,psil,pol,xFe,xMg,ph2o];
  
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 12. - 0.4*P
    W[1] = -2. - 0.5*P
    W[2] = 5.
    W[3] = 12.
    W[4] = 12. - 0.4*P
    W[5] = 14.
    W[6] = 17. - 0.5*P
    W[7] = -6. + 3.*P
    W[8] = 0.
    W[9] = 12.
    W[10] = 10.
    W[11] = 2.
    W[12] = -1.5 - 0.3*P
    W[13] = 0. - 1.*P
    W[14] = 12.
    W[15] = 12.
    W[16] = 12.
    W[17] = 9.5 - 0.3*P
    W[18] = 0.
    W[19] = 0.
    W[20] = 0.
    W[21] = 7.5 - 0.5*P
    W[22] = 12.
    W[23] = 12.
    W[24] = 11.
    W[25] = 18.
    W[26] = 11 - 0.5*P
    W[27] = 12.
    v    = 0.
    
    idm[0]  =   sympify('fac*pq');
    idm[1]  =   sympify('fac*xab');
    idm[2]  =   sympify('fac*xksp');
    idm[3]  =   sympify('fac*pan');
    idm[4]  =   sympify('fac*psil');
    idm[5]  =   sympify('fac*pol*xMg**5');
    idm[6]  =   sympify('fac*pol*xFe**5');
    idm[7]  =   sympify('ph2o**2');

    sym_sf = ['fac', 'pq', 'xab', 'xksp', 'pan', 'psil', 'pol', 'xFe', 'xMg', 'ph2o'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');
    gbase[6]  = sympify('gb6');
    gbase[7]  = sympify('gb7');

    symb    = ['q','fsp','na','an','ol','xc','h2o']
    
    cv[0] = 'q   = p0';
    cv[1] = 'an  = p3';
    cv[2] = 'h2o = p7';
    cv[3] = 'fsp = p1 + p2';
    cv[4] = 'na  = p1/fsp';
    cv[5] = 'ol  = 1.0 - q - fsp - an - h2o - p4';
    cv[6] = 'xc   = p6/ol';

    
    in_var  = ['q','fsp','na','an','ol','xc','h2o','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','ln']
    in_sf   = ['fac', 'pq', 'xab', 'xksp', 'pan', 'psil', 'pol', 'xFe', 'xMg', 'ph2o'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def pl4tr_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 3

    W         = [0]*3
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.049;
    IG       = [0]*(n_em-1);
    cv       = [0]*(n_em-1);
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    
    emname    = ['ab','an','san']
    make      = [0]*n_em
    make[0]   = sympify('ab')
    make[1]   = sympify('an')
    make[2]   = sympify('san')

    
    p[0] = sympify('1-k-ca');
    p[1] = sympify('ca');
    p[2] = sympify('k');

    
    #     Site fractions
    xNaA = sympify('1 - ca - k');
    xCaA =sympify(' ca');
    xKA = sympify('k');
    xAlTB = sympify('1/4 + 1/4*ca');
    xSiTB = sympify('3/4 - 1/4*ca');
    
    sf = [xNaA,xCaA,xKA,xAlTB,xSiTB];
  
    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 14.6 - 0.00935*T - 0.04*P
    W[1] = 24.1 - 0.00957*T + 0.338*P
    W[2] = 48.5 - 0.13*P
    v[0] = 0.674
    v[1] = 0.55
    v[2] = 1.0
    
    idm[0]  =   sympify('1.7548*xNaA*xAlTB**(1/4)*xSiTB**(3/4)');
    idm[1]  =   sympify('2*xCaA*xAlTB**(1/2)*xSiTB**(1/2)');
    idm[2]  =   sympify('1.7548*xKA*xAlTB**(1/4)*xSiTB**(3/4)');

    sym_sf = ['xNaA','xCaA','xKA','xAlTB','xSiTB'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');

    symb    = ['ca','k']
    
    cv[0] = 'k   = p2';
    cv[1] = 'ca  = p1';

    in_var  = ['ca','k','log']   
    out_var = ['x[0]','x[1]','ln']
    in_sf   = ['xNaA','xCaA','xKA','xAlTB','xSiTB'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def bi_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 7

    W         = [0]*21
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.19;
    IG       = [0]*(n_em-1);
    cv       = [0]*(n_em-1);
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1]; 
    Range[3]  = [0,1];
    Range[4]  = [0,1];
    Range[5]  = [0,1];
    
    emname    = ['phl','annm','obi','east','tbi','fbi','mmbi']
    make      = [0]*n_em 
    make[0]   = sympify('phl');
    make[1]   = sympify('ann - 3');
    make[2]   = sympify('1/3* ann + 2/3 *phl - 3');
    make[3]   = sympify('east');
    make[4]   = sympify('phl - br + ru + 55');
    make[5]   = sympify('east - 1/2* gr + 1/2* andr - 3');
    make[6]   = sympify('mnbi - 7.89');

    p[0] = sympify('1 - f - m - t - xc - y - 2/3*q + f*xc + 3*m*xc + t*xc + xc*y');
    p[1] = sympify('-1/3*q + xc');
    p[2] = sympify('q + (-f)*xc + (-3*m)*xc + (-t)*xc + (-xc)*y');
    p[3] = sympify('y');
    p[4] = sympify('t');
    p[5] = sympify('f');
    p[6] = sympify('m');
    
    #     Site fractions
    xMgM3  = sympify(' 1 - f - m - t - xc - y - 2/3*q + f*xc + 3*m*xc + t*xc + xc*y');
    xMnM3  = sympify('m');
    xFeM3  = sympify('xc + 2/3*q + (-f)*xc + (-3*m)*xc + (-t)*xc + (-xc)*y');
    xFe3M3 = sympify('f');
    xTiM3  = sympify('t');
    xAlM3  = sympify('y');
    xMgM12 = sympify('1 + 1/3*q - m - xc');
    xMnM12 = sympify('m');
    xFeM12 = sympify('-1/3*q + xc');
    xSiT   = sympify('1/2 - 1/2*f - 1/2*y');
    xAlT   = sympify('1/2 + 1/2*f + 1/2*y');
    xOHV   = sympify('1 - t');
    xOV    = sympify('t');

    sf = [xMgM3,xMnM3,xFeM3,xFe3M3,xTiM3,xAlM3,xMgM12,xMnM12,xFeM12,xSiT,xAlT,xOHV,xOV];
  
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 12
    W[1] = 4
    W[2] = 10
    W[3] = 30
    W[4] = 8
    W[5] = 9
    W[6] = 8
    W[7] = 15
    W[8] = 32
    W[9] = 13.6
    W[10] = 6.3
    W[11] = 7
    W[12] = 24
    W[13] = 5.6
    W[14] = 8.1
    W[15] = 40
    W[16] = 1
    W[17] = 13
    W[18] = 40
    W[19] = 30
    W[20] = 11.6
    v    = 0.
    
    idm[0] = sympify('4*xMgM3 *xMgM12**2 *xSiT *xAlT *xOHV**2');
    idm[1] = sympify('4*xFeM3 *xFeM12**2 *xSiT *xAlT *xOHV**2');
    idm[2] = sympify('4*xFeM3 *xMgM12**2 *xSiT *xAlT *xOHV**2');
    idm[3] = sympify('xAlM3 *xMgM12**2 *xAlT**2 *xOHV**2');
    idm[4] = sympify('4*xTiM3 *xMgM12**2 *xSiT *xAlT *xOV**2');
    idm[5] = sympify('xFe3M3 *xMgM12**2 *xAlT**2 *xOHV**2');
    idm[6] = sympify('4*xMnM3 *xMnM12**2 *xSiT *xAlT *xOHV**2');


    sym_sf = ['xMgM3','xMnM3','xFeM3','xFe3M3','xTiM3','xAlM3','xMgM12','xMnM12','xFeM12','xSiT','xAlT','xOHV','xOV'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');
    gbase[6]  = sympify('gb6');

    symb    = ['xc','m','y','f','t','q']
    
    cv[0]   = 'm  = p6';
    cv[1]   = 'f  = p5';
    cv[2]   = 't  = p4';
    cv[3]   = 'y  = p3';
    cv[4]   = 'q  = 3.0*(-f*p1 + f - 3.0*m*p1 + m + p0 - p1*t - p1*y + p1 + t + y - 1.0)/(f + 3.0*m + t + y - 3.0)';
    cv[5]   = 'xc  = (-p2 + q)/(f + 3.0*m + t + y)';
      
    in_var  = ['xc','m','y','f','t','q','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','ln']
    
    in_sf   =  ['xMgM3','xMnM3','xFeM3','xFe3M3','xTiM3','xAlM3','xMgM12','xMnM12','xFeM12','xSiT','xAlT','xOHV','xOV'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]','sf[10]','sf[11]','sf[12]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def g_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 5

    W         = [0]*10
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.19;
    IG        = [0]*(n_em-1);
    cv        = [0]*(n_em-1);
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1]; 
    Range[3]  = [0,1]; 
    
    emname    = ['py','alm','spss','gr','kho']
    make      = [0]*n_em
    make[0]   = sympify('py')
    make[1]   = sympify('alm')
    make[2]   = sympify('spss')
    make[3]   = sympify('gr')
    make[4]   = sympify('py - gr + andr + 27')
    
    
    p[0] = sympify('1 - f - m - xc - z + m*xc + xc*z');
    p[1] = sympify('xc + (-m) *xc + (-xc) *z');
    p[2] = sympify('m');
    p[3] = sympify('z');
    p[4] = sympify('f');

    
    #     Site fractions
    xMgX = sympify('1 - m - xc - z + m*xc + xc*z');
    xFeX = sympify('xc + (-m) *xc + (-xc) *z');
    xMnX = sympify('m');
    xCaX = sympify('z');
    xAlY = sympify('1 - f');
    xFe3Y = sympify('f');

    sf = [xMgX,xFeX,xMnX,xCaX,xAlY,xFe3Y];
  
    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0]  = 2.5
    W[1] = 2.0
    W[2] = 31.0
    W[3] = 5.4
    W[4] = 2.0
    W[5] = 5.0
    W[6] = 22.6
    W[7] = 0.0
    W[8] = 29.4
    W[9] = -15.3
    
    v[0] = 1.0
    v[1] = 1.0
    v[2] = 1.0
    v[3] = 2.7
    v[4] = 1.0

    
    idm[0] = sympify('xMgX**3 *xAlY**2');
    idm[1] = sympify('xFeX**3 *xAlY**2');
    idm[2] = sympify('xMnX**3 *xAlY**2');
    idm[3] = sympify('xCaX**3 *xAlY**2');
    idm[4] = sympify('xMgX**3 *xFe3Y**2');


    sym_sf = ['xMgX','xFeX','xMnX','xCaX','xAlY','xFe3Y'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');

    symb    = ['xc','z','m','f']
    
    cv[0]   = 'f   =  p4';
    cv[1]   = 'z   =  p3';
    cv[2]   = 'm   =  p2';
    cv[3]   = 'xc  =  p1/(1.0 - m - z)';

    in_var  = ['xc','z','m','f','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','ln']
    
    in_sf   =  ['xMgX','xFeX','xMnX','xCaX','xAlY','xFe3Y'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def ep_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 3

    W         = [0]*3
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.049;
    # Initial guess -> ca,k
    IG       = [0]*(n_em-1);
    cv       = [0]*(n_em-1);
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,0.5];
    
    emname    = ['cz','ep','fep']
    make      = [0]*n_em
    make[0]   = sympify('cz')
    make[1]   = sympify('ep')
    make[2]   = sympify('fep')


    p[0] = sympify(' 1 - f - q');
    p[1] = sympify(' 2*q');
    p[2] = sympify(' f - q');

    #     Site fractions
    xFeM1 = sympify('f - q');
    xAlM1 = sympify('1 - f + q');
    xFeM3 = sympify('f + q');
    xAlM3 = sympify('1 - f - q');
    
    sf = [xFeM1,xAlM1,xFeM3,xAlM3];
  
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 1
    W[1] = 3
    W[2] = 1
    v    = 0.
    
    idm[0] = sympify('xAlM1 *xAlM3');
    idm[1] = sympify('xAlM1 *xFeM3');
    idm[2] = sympify('xFeM1 *xFeM3');

    sym_sf = ['xFeM1','xAlM1','xFeM3','xAlM3'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');

    symb    = ['f','q']
    
    cv[0]   = 'q   =  p1/2.0';
    cv[1]   = 'f   =  p2 + q';
    
    in_var  = ['f','q','log']   
    out_var = ['x[0]','x[1]','ln']
    in_sf   = ['xFeM1','xAlM1','xFeM3','xAlM3'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def ma_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 6

    W         = [0]*15
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.249;
    IG       = [0]*(n_em-1);
    cv       = [0]*(n_em-1);
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1]; 
    Range[3]  = [0,1];
    Range[4]  = [0,1];
    
    emname    = ['mut','celt','fcelt','pat','ma','fmu']
    make      = [0]*n_em
    make[0]   = sympify('mu + 1')
    make[1]   = sympify('cel + 5')
    make[2]   = sympify('fcel + 5')
    make[3]   = sympify('pa + 4')
    make[4]   = sympify('ma')
    make[5]   = sympify('mu - 1/2 *gr + 1/2 *andr + 25')

    
    p[0] = sympify('-c - f - n + y');
    p[1] = sympify('1 - xc - y + xc *y');
    p[2] = sympify('xc + (-xc) *y');
    p[3] = sympify('n');
    p[4] = sympify('c');
    p[5] = sympify('f');
    
    #     Site fractions
    xKA     = sympify('1 - c - n');
    xNaA    = sympify(' n');
    xCaA    = sympify('c');
    xMgM2A  = sympify('1 - xc - y + xc *y');
    xFeM2A  = sympify('xc + (-xc) *y');
    xAlM2A  = sympify('y');
    xAlM2B  = sympify('1 - f');
    xFe3M2B = sympify('f');
    xSiT1   = sympify('1 - 1/2 *c - 1/2 *y');
    xAlT1   = sympify('1/2 *c + 1/2 *y');

    sf = [xKA,xNaA,xCaA,xMgM2A,xFeM2A,xAlM2A,xAlM2B,xFe3M2B,xSiT1,xAlT1];
  
    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 0 + 0.2 *P
    W[1] = 0 + 0.2 *P
    W[2] = 10.12 + 0.0034 *T + 0.353 *P
    W[3] = 34
    W[4] = 0
    W[5] = 0
    W[6] = 45 + 0.25 *P
    W[7] = 50
    W[8] = 0
    W[9] = 45 + 0.25 *P
    W[10] = 50
    W[11] = 0
    W[12] = 18
    W[13] = 30
    W[14] = 35

    v[0] = 0.63
    v[1] = 0.63
    v[2] = 0.63
    v[3] = 0.37
    v[4] = 0.63
    v[5] = 0.63

    idm[0]  = sympify('4*xKA *xAlM2A *xAlM2B *xSiT1 *xAlT1');
    idm[1]  = sympify('xKA *xMgM2A *xAlM2B *xSiT1**2');
    idm[2]  = sympify('xKA *xFeM2A *xAlM2B *xSiT1**2');
    idm[3]  = sympify('4*xNaA *xAlM2A *xAlM2B *xSiT1 *xAlT1');
    idm[4]  = sympify('xCaA *xAlM2A *xAlM2B *xAlT1**2');
    idm[5]  = sympify('4*xKA *xAlM2A *xFe3M2B *xSiT1 *xAlT1');

    sym_sf = ['xKA','xNaA','xCaA','xMgM2A','xFeM2A','xAlM2A','xAlM2B','xFe3M2B','xSiT1','xAlT1'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');

    symb    = ['xc','y','f','n','c']
    
    cv[0]   = 'f  = p5';
    cv[1]   = 'c  = p4';
    cv[2]   = 'n  = p3';
    cv[3]   = 'y  = p0 + c + n + f';
    cv[4]   = 'xc = p2/(1.0-y)';
      
    in_var  = ['xc','y','f','n','c','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','ln']
    
    in_sf   =  ['xKA','xNaA','xCaA','xMgM2A','xFeM2A','xAlM2A','xAlM2B','xFe3M2B','xSiT1','xAlT1'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]']

    return symb,cv,  p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def mu_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 6

    W         = [0]*15
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.249;
    IG        = [0]*(n_em-1);
    cv        = [0]*(n_em-1);
   
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1]; 
    Range[3]  = [0,1];
    Range[4]  = [0,1];
    
    emname    = ['mut','cel','fcel','pat','ma','fmu']
    make      = [0]*n_em
    make[0]   = sympify('mu')
    make[1]   = sympify('cel')
    make[2]   = sympify('fcel')
    make[3]   = sympify('pa')
    make[4]   = sympify('ma + 5')
    make[5]   = sympify('1/2 *andr - 1/2 *gr + mu + 25')

    p[0] = sympify('-c - f - n + y');
    p[1] = sympify('1 - xc - y + xc *y');
    p[2] = sympify('xc + (-xc) *y');
    p[3] = sympify('n');
    p[4] = sympify('c');
    p[5] = sympify('f');
    
    #     Site fractions
    xKA = sympify('1 - c - n');
    xNaA =sympify(' n');
    xCaA = sympify('c');
    xMgM2A = sympify('1 - xc - y + xc* y');
    xFeM2A = sympify('xc + (-xc) *y');
    xAlM2A = sympify('y');
    xAlM2B = sympify('1 - f');
    xFe3M2B = sympify('f');
    xSiT1 = sympify('1 - 1/2 *c - 1/2 *y');
    xAlT1 = sympify('1/2 *c + 1/2 *y');


    sf = [xKA,xNaA,xCaA,xMgM2A,xFeM2A,xAlM2A,xAlM2B,xFe3M2B,xSiT1,xAlT1];
  
    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 0 + 0.2 *P
    W[1] = 0 + 0.2 *P
    W[2] = 10.12 + 0.0034 *T + 0.353 *P
    W[3] = 35
    W[4] = 0
    W[5] = 0
    W[6] = 45 + 0.25 *P
    W[7] = 50
    W[8] = 0
    W[9] = 45 + 0.25 *P
    W[10] = 50
    W[11] = 0
    W[12] = 15
    W[13] = 30
    W[14] = 35

    v[0] = 0.63
    v[1] = 0.63
    v[2] = 0.63
    v[3] = 0.37
    v[4] = 0.63
    v[5] = 0.63

    idm[0]  = sympify('4*xKA *xAlM2A *xAlM2B *xSiT1 *xAlT1');
    idm[1]  = sympify('xKA *xMgM2A *xAlM2B *xSiT1**2');
    idm[2]  = sympify('xKA *xFeM2A *xAlM2B *xSiT1**2');
    idm[3]  = sympify('4*xNaA *xAlM2A *xAlM2B *xSiT1 *xAlT1');
    idm[4]  = sympify('xCaA *xAlM2A *xAlM2B *xAlT1**2');
    idm[5]  = sympify('4*xKA *xAlM2A *xFe3M2B *xSiT1 *xAlT1');

    sym_sf = ['xKA','xNaA','xCaA','xMgM2A','xFeM2A','xAlM2A','xAlM2B','xFe3M2B','xSiT1','xAlT1'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');

    symb    = ['xc','y','f','n','c']
    
    cv[0]   = 'f  = p5';
    cv[1]   = 'c  = p4';
    cv[2]   = 'n  = p3';
    cv[3]   = 'y  = p0 + c + n + f';
    cv[4]   = 'xc = p2/(1.0-y)';
      
    in_var  = ['xc','y','f','n','c','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','ln']
    
    in_sf   =  ['xKA','xNaA','xCaA','xMgM2A','xFeM2A','xAlM2A','xAlM2B','xFe3M2B','xSiT1','xAlT1'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def opx_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 7

    W         = [0]*21
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.249;
    # Initial guess -> ca,k
    IG        = [0]*(n_em-1);
    cv        = [0]*(n_em-1);
        
    # Validity bounds
    Range     = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,2]; 
    Range[3]  = [0,1];
    Range[4]  = [0,1];
    Range[5]  = [0,1];
    
    emname    = ['en','fs','fm','mgts','fopx','mnopx','odi']
    make      = [0]*n_em
    make[0]   = sympify('en')
    make[1]   = sympify('fs')
    make[2]   = sympify('1/2 *en + 1/2 *fs - 6.6')
    make[3]   = sympify('mgts')
    make[4]   = sympify('mgts - 1/2* gr + 1/2 *andr + 2')
    make[5]   = sympify('2* pxmn + 6.68')
    make[6]   = sympify('di - 0.1 + 0.000211 *T + 0.005 *P')

    p[0] = sympify('1 - 1/2* q - c - f - m - xc - y + 1/2 *c* q + 1/2 *m *q + c* xc + m *xc');
    p[1] = sympify('-1/2* q + xc + 1/2 *c* q + 1/2* m *q + (-f)* xc + (-m)* xc + (-xc) *y');
    p[2] = sympify('q + (-c) *q + (-m) *q + (-c) *xc + f* xc + xc* y');
    p[3] = sympify('y');
    p[4] = sympify('f');
    p[5] = sympify('m');
    p[6] = sympify('c');
    
    #     Site fractions
    xMgM1 = sympify('1 + 1/2 *q - f - m - xc - y + (-1/2 *c) *q + (-1/2 *m)* q + f *xc + m*xc + xc *y');
    xFeM1 = sympify('-1/2* q + xc + 1/2* c *q + 1/2* m *q + (-f)* xc + (-m)* xc + (-xc)*y');
    xMnM1 = sympify('m');
    xFe3M1= sympify('f');
    xAlM1 = sympify('y');
    xMgM2 = sympify('1 - 1/2 *q - c - m - xc + 1/2 *c* q + 1/2 *m *q + c* xc + m*xc');
    xFeM2 = sympify('1/2*q + xc + (-1/2 *c) *q + (-1/2 *m)* q + (-c) *xc + (-m)*xc');
    xMnM2 = sympify('m');
    xCaM2 = sympify('c');
    xSiT  = sympify('1 - 1/2* f - 1/2 *y');
    xAlT  = sympify('1/2 *f + 1/2* y');
    sf    = [xMgM1,xFeM1,xMnM1,xFe3M1,xAlM1,xMgM2,xFeM2,xMnM2,xCaM2,xAlT,xSiT];
  
    # Non-ideality by symmetric formalism
    symmetry = 0;

    W[0] = 7.
    W[1] = 4.
    W[2] = 13. - 0.15 *P
    W[3] = 11. - 0.15* P
    W[4] = 5.
    W[5] = 32.2 + 0.12 *P
    W[6] = 4.
    W[7] = 13. - 0.15 *P
    W[8] = 11.6 - 0.15 *P
    W[9] = 4.2
    W[10] = 25.54 + 0.084* P
    W[11] = 17. - 0.15* P
    W[12] = 15. - 0.15 *P
    W[13] = 5.1
    W[14] = 22.54 + 0.084 *P
    W[15] = 1.
    W[16] = 12. - 0.15 *P
    W[17] = 75.4 - 0.94 *P
    W[18] = 10.6 - 0.15 *P
    W[19] = 73.4 - 0.94 *P
    W[20] = 24.54 + 0.084* P

    v[0] = 1.
    v[1] = 1.
    v[2] = 1.
    v[3] = 1.
    v[4] = 1.
    v[5] = 1.
    v[6] = 1.2

    idm[0] = sympify('xMgM1 *xMgM2 *xSiT**(1/2)');
    idm[1] = sympify('xFeM1 *xFeM2 *xSiT**(1/2)');
    idm[2] = sympify('xMgM1 *xFeM2 *xSiT**(1/2)');
    idm[3] = sympify('1.4142 *xAlM1 *xMgM2 *xAlT**(1/4) *xSiT**(1/4)');
    idm[4] = sympify('1.4142 *xFe3M1 *xMgM2* xAlT**(1/4) *xSiT**(1/4)');
    idm[5] = sympify('xMnM1 *xMnM2 *xSiT**(1/2)');
    idm[6] = sympify('xMgM1 *xCaM2 *xSiT**(1/2)');

    sym_sf = ['xMgM1','xFeM1','xMnM1','xFe3M1','xAlM1','xMgM2','xFeM2','xMnM2','xCaM2','xAlT','xSiT'];
  
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');
    gbase[6]  = sympify('gb6');
    
    symb    = ['xc','m','y','f','c','q']
    
    cv[0]   = 'c  = p6';
    cv[1]   = 'm  = p5';
    cv[2]   = 'f  = p4';
    cv[3]   = 'y  = p3';
    cv[4]   = 'xc = (-2.0*p1 - p2)/(c + f + 2.0*m + y - 2.0)';
    cv[5]   = 'q  = 2.0*(pow(c, 2) + 2.0*c*f + 3.0*c*m + c*p0 + 2.0*c*p1 + c*p2 + 2.0*c*y - 3.0*c + pow(f, 2) + 3.0*f*m + f*p0 + 2.0*f*y - 3.0*f + 2.0*pow(m, 2) + 2.0*m*p0 + 2.0*m*p1 + m*p2 + 3.0*m*y - 4.0*m + p0*y - 2.0*p0 - 2.0*p1 - p2 + pow(y, 2) - 3.0*y + 2.0)/(pow(c, 2) + c*f + 3.0*c*m + c*y - 3.0*c + f*m - f + 2.0*pow(m, 2) + m*y - 4.0*m - y + 2.0)';
      
    in_var  = ['xc','m','y','f','c','q','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','ln']
    
    in_sf   =  ['xMgM1','xFeM1','xMnM1','xFe3M1','xAlM1','xMgM2','xFeM2','xMnM2','xCaM2','xAlT','xSiT'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]','sf[10]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def sa_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 5

    W         = [0]*10
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.19;
    IG        = [0]*(n_em-1);
    cv        = [0]*(n_em-1);
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1]; 
    Range[3]  = [-1,1];
    
    emname    = ['spr4','spr5','fspm','spro','ospr']
    make      = [0]*n_em
    make[0]   = sympify('spr4');
    make[1]   = sympify('spr5');
    make[2]   = sympify('fspr - 2');
    make[3]   = sympify('3/4 *fspr + 1/4 *spr4 - 3.5');
    make[4]   = sympify('1/2 *andr - 1/2 *gr + spr5 - 16');
    
    p[0] = sympify('1 - 1/4 *q - f - xc - y');
    p[1] = sympify('y');
    p[2] = sympify('xc - 3/4* q + (-f) *xc + (-xc)* y');
    p[3] = sympify('q + f *xc + xc *y');
    p[4] = sympify('f');

    #     Site fractions
    xMgM3   = sympify('1 - f - xc - y + 3/4 *q + f *xc + xc *y');
    xFeM3   = sympify('xc - 3/4* q + (-f)* xc + (-xc)* y');
    xFe3M3  = sympify('f');
    xAlM3   = sympify('y');
    xMgM456 = sympify('1 - 1/4* q - xc');
    xFeM456 = sympify('1/4 *q + xc');
    xSiT    = sympify('1 - f - y');
    xAlT    = sympify('f + y');


    sf = [xMgM3,xFeM3,xFe3M3,xAlM3,xMgM456,xFeM456,xSiT,xAlT];
  
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 10 - 0.02 *P
    W[1] = 16
    W[2] = 12
    W[3] = 8 - 0.02 *P
    W[4] = 19 - 0.02 *P
    W[5] = 22 - 0.02 *P
    W[6] = 1
    W[7] = 4
    W[8] = 17.6 - 0.02 *P
    W[9] = 20 - 0.02 *P

    v = 0.0

    idm[0] = sympify('xMgM3 *xMgM456**3 *xSiT');
    idm[1] = sympify('xAlM3 *xMgM456**3 *xAlT');
    idm[2] = sympify('xFeM3 *xFeM456**3 *xSiT');
    idm[3] = sympify('xMgM3 *xFeM456**3 *xSiT');
    idm[4] = sympify('xFe3M3 *xMgM456**3 *xAlT');

    sym_sf = ['xMgM3','xFeM3','xFe3M3','xAlM3','xMgM456','xFeM456','xSiT','xAlT'];
  
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');

    symb    = ['xc','y','f','q']
    
    cv[0]   = 'f  = p4';
    cv[1]   = 'y  = p1';
    cv[2]   = 'xc = (4.0*f + 4.0*p0 + p3 + 4.0*y - 4.0)/(f + y - 4.0)';
    cv[3]   = 'q  = 1.3333333333333333*(-4.0*pow(f, 2) - 4.0*f*p0 - f*p2 - f*p3 - 8.0*f*y + 8.0*f - 4.0*p0*y + 4.0*p0 - p2*y + 4.0*p2 - p3*y + p3 - 4.0*pow(y, 2) + 8.0*y - 4.0)/(f + y - 4.0)';
     
    in_var  = ['xc','y','f','q','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','ln']
    
    in_sf   =  ['xMgM3','xFeM3','xFe3M3','xAlM3','xMgM456','xFeM456','xSiT','xAlT'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def cd_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 4

    W         = [0]*6
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.19;
    IG        = [0]*(n_em-1);
    cv        = [0]*(n_em-1);
    
    # Validity bounds
    Range     = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1];
    
    emname    = ['crd','fcrd','hcrd','mncd']
    make      = [0]*n_em
    make[0]   = sympify('crd');
    make[1]   = sympify('fcrd');
    make[2]   = sympify('hcrd');
    make[3]   = sympify('e_mncrd - 4.21');

    
    p[0] = sympify('1 - h - m - xc + m* xc');
    p[1] = sympify('xc + (-m) *xc');
    p[2] = sympify('h');
    p[3] = sympify('m');
    
    #     Site fractions
    xFeX  = sympify('xc');
    xMgX  = sympify('1 - xc');
    xMnX  = sympify('m');
    xH2OH = sympify('h');
    xvH   = sympify('1 - h');
    
    sf = [xFeX,xMgX,xMnX,xH2OH,xvH];
  
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 8
    W[1] = 0
    W[2] = 6
    W[3] = 9
    W[4] = 7
    W[5] = 6
    v    = 0.
    
    idm[0] = sympify('xMgX**2 *xvH');
    idm[1] = sympify('xFeX**2 *xvH');
    idm[2] = sympify('xMgX**2 *xH2OH');
    idm[3] = sympify('xMnX**2 *xvH');
    
    sym_sf = ['xFeX','xMgX','xMnX','xH2OH','xvH'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    
    symb    = ['xc','m','h']

    cv[0]   = 'm  = p3';
    cv[1]   = 'h  = p2';
    cv[2]   = 'xc = p1/(1.0 - m)';

    in_var  = ['xc','m','h','log']   
    out_var = ['x[0]','x[1]','x[2]','ln']
    in_sf   = ['xFeX','xMgX','xMnX','xH2OH','xvH'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def st_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 5

    W         = [0]*10
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.19;
    # Initial guess -> ca,k
    IG       = [0]*(n_em-1);
    cv       = [0]*(n_em-1);
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1]; 
    Range[3]  = [0,1]; 
    
    emname    = ['mstm','fst','mnstm','msto','mstt']
    make      = [0]*n_em
    make[0]   = sympify('mst - 8.0');
    make[1]   = sympify('fst');
    make[2]   = sympify('mnst - 0.19');
    make[3]   = sympify('mst - gr + andr + 9.0');
    make[4]   = sympify('mst - cor + 3/2 *ru + 13.0');
    
    p[0] = sympify('1.0 - f - m - xc - 4.0/3.0* t + m *xc');
    p[1] = sympify('xc + (-m)* xc');
    p[2] = sympify('m');
    p[3] = sympify('f');
    p[4] = sympify('4.0/3.0* t');

    #     Site fractions
    xMgX = sympify('1.0 - m - xc + m *xc');
    xFeX = sympify('xc + (-m)* xc');
    xMnX = sympify('m');
    xAlY = sympify('1.0 - f - 4.0/3.0* t');
    xFe3Y= sympify('f');
    xTiY = sympify('t');
    xvY  = sympify('1.0/3.0 *t');

    sf    = [xMgX,xFeX,xMnX,xAlY,xFe3Y,xTiY,xvY];
  
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 16.0
    W[1] = 12.0
    W[2] = 2.0
    W[3] = 20.0
    W[4] = 8.0
    W[5] = 18.0
    W[6] = 36.0
    W[7] = 14.0
    W[8] = 32.0
    W[9] = 30.0
    v    = 0.0

    idm[0] = sympify('xMgX**4.0 *xAlY**2.0');
    idm[1] = sympify('xFeX**4.0 *xAlY**2.0');
    idm[2] = sympify('xMnX**4.0 *xAlY**2.0');
    idm[3] = sympify('xMgX**4.0* xFe3Y**2.0');
    idm[4] = sympify('3.0792 *xMgX**4.0 *xTiY**(3.0/2.0) *xvY**(1.0/2.0)');

    sym_sf = ['xMgX','xFeX','xMnX','xAlY','xFe3Y','xTiY','xvY'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    
    symb    = ['xc','m','f','t']
    
    cv[0]   = 't  = p4/(4.0/3.0)';
    cv[1]   = 'f  = p3';
    cv[2]   = 'm  = p2';
    cv[3]   = 'xc = p1/(1.0 - m)';

    in_var  = ['xc','m','f','t','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','ln']
    
    in_sf   = ['xMgX','xFeX','xMnX','xAlY','xFe3Y','xTiY','xvY'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def chl_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 8

    W         = [0]*28
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.249;
    IG        = [0]*(n_em-1);
    cv        = [0]*(n_em-1);
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1]; 
    Range[3]  = [0,1];
    Range[4]  = [-1,1];
    Range[5]  = [-1,1];
    Range[6]  = [-1,1];
    
    emname    = ['clin','afchl','ames','daph','ochl1','ochl4','f3clin','mmchl']
    make      = [0]*n_em
    make[0]   = sympify('clin');
    make[1]   = sympify('afchl');
    make[2]   = sympify('ames');
    make[3]   = sympify('daph');
    make[4]   = sympify('afchl - clin + daph + 3.0');
    make[5]   = sympify('afchl - 1.0/5.0 *clin + 1/5* daph + 2.4');
    make[6]   = sympify('clin - 1.0/2.0* gr + 1.0/2.0 *andr + 2.0');
    make[7]   = sympify('mnchl - 5.67');

    
    p[0] = sympify('-1/4 *q1 - m + 2*qal - 5/4 *q4 + 1/4 *m *q1 + 5/4* f *q4 + (-1/4 *q1)* qal + 5/4 *q4 *qal + (-f) *xc + m* xc + (-qal)* xc + 1/4 *q1 *y + 5/4* q4* y + (-xc)* y');
    p[1] = sympify('1 - f - qal - y - 2*xc + 5/4 *q1 + 9/4 *q4 + (-5/4 *m)* q1 + (-9/4* f) *q4 + 5/4 *q1* qal + (-9/4 *q4)* qal + 2*f* xc + qal* xc + (-5/4* q1) *y + (-9/4 *q4)* y + 3*xc *y');
    p[2] = sympify('-qal + y');
    p[3] = sympify('1/4 *q1 + 5/4 *q4 + (-1/4 *m) *q1 + (-5/4* f) *q4 + 1/4* q1* qal + (-5/4 *q4)* qal + f* xc + (-m)* xc + qal* xc + (-1/4* q1)* y + (-5/4 *q4)* y + xc *y');
    p[4] = sympify('-q4 + xc + f *q4 + q4 *qal + (-f) *xc + (-qal)* xc + q4 *y + (-xc)* y');
    p[5] = sympify('xc - 5/4 *q1 - 5/4 *q4 + 5/4* m *q1 + 5/4* f *q4 + (-5/4 *q1)* qal + 5/4 *q4 *qal + (-f)* xc + 5/4* q1* y + 5/4 *q4 *y + (-2*xc) *y');
    p[6] = sympify('f');
    p[7] = sympify('m');
    
    #     Site fractions
    xMgM1 = sympify('1 - m + q1 + qal - xc - y + (-m)* q1 + q1 *qal + m *xc + (-qal)*xc + (-q1) *y + xc* y');
    xMnM1 = sympify('m');
    xFeM1 = sympify('-q1 + xc + m* q1 + (-q1)* qal + (-m)* xc + qal* xc + q1 *y + (-xc)* y');
    xAlM1 = sympify('-qal + y');
    xMgM23 = sympify('1 - 1/4 *q1 - 1/4* q4 - m - xc + 1/4 *m *q1 + 1/4 *f *q4 + (-1/4 *q1) *qal + 1/4 *q4 *qal + m *xc + 1/4 *q1* y + 1/4 *q4* y');
    xMnM23 = sympify('m');
    xFeM23 = sympify('1/4* q1 + 1/4 *q4 + xc + (-1/4* m)* q1 + (-1/4 *f) *q4 + 1/4 *q1* qal + (-1/4 *q4)* qal + (-m)* xc + (-1/4 *q1)* y + (-1/4* q4)* y');
    xMgM4 = sympify('1 - f + q4 - qal - xc - y + (-f)* q4 + (-q4) *qal + f *xc + qal *xc + (-q4)* y + xc *y');
    xFeM4 = sympify('-q4 + xc + f* q4 + q4* qal + (-f) *xc + (-qal)* xc + q4* y + (-xc)* y');
    xFe3M4 = sympify('f');
    xAlM4 = sympify('qal + y');
    xSiT2 = sympify('1 - 1/2 *f - y');
    xAlT2 = sympify('1/2* f + y');

    # Here the xMnM1 sitefraction is not considered as it is redondant with xMnM23
    sf = [xMgM1,xFeM1,xAlM1,xMgM23,xMnM23,xFeM23,xMgM4,xFeM4,xFe3M4,xAlM4,xSiT2,xAlT2];
  
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 17.
    W[1] = 17.
    W[2] = 20.
    W[3] = 30.
    W[4] = 21.
    W[5] = 2.
    W[6] = 6.
    W[7] = 16.
    W[8] = 37.
    W[9] = 20.
    W[10] = 4.
    W[11] = 15.
    W[12] = 23.
    W[13] = 30.
    W[14] = 29.
    W[15] = 13.
    W[16] = 19.
    W[17] = 17.
    W[18] = 18.
    W[19] = 33.
    W[20] = 22.
    W[21] = 4.
    W[22] = 24.
    W[23] = 28.6
    W[24] = 19.
    W[25] = 19.
    W[26] = 22.
    W[27] = 8.

    v    = 0.
    
    idm[0] = sympify('4*xMgM1 *xMgM23**4 *xAlM4 *xSiT2 *xAlT2');
    idm[1] = sympify('xMgM1 *xMgM23**4 *xMgM4 *xSiT2**2');
    idm[2] = sympify('xAlM1 *xMgM23**4 *xAlM4 *xAlT2**2');
    idm[3] = sympify('4*xFeM1 *xFeM23**4 *xAlM4* xSiT2 *xAlT2');
    idm[4] = sympify('xMgM1 *xFeM23**4 *xFeM4 *xSiT2**2');
    idm[5] = sympify('xFeM1 *xMgM23**4 *xMgM4 *xSiT2**2');
    idm[6] = sympify('4*xMgM1 *xMgM23**4 *xFe3M4 *xSiT2 *xAlT2');
    idm[7] = sympify('4*xMnM23 *xMnM23**4 *xAlM4* xSiT2* xAlT2');
    
    sym_sf = ['xMgM1','xFeM1','xAlM1','xMgM23','xMnM23','xFeM23','xMgM4','xFeM4','xFe3M4','xAlM4','xSiT2','xAlT2'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    gbase[5]  = sympify('gb5');
    gbase[6]  = sympify('gb6');
    gbase[7]  = sympify('gb7');

    symb    = ['xc','y','f','m','qal','q1','q4']
    
    cv[0] = 'f   =  p6';
    cv[1] = 'm   =  p7';
    cv[2] = 'qal = (m + 1.0 - f -(p1 - p3+p5-p0 + p2+p4))/4.0';
    cv[3] = 'y   =  p2 + qal';
    cv[4] = 'xc  = (-2.0*qal + m + p0 - 4.0*p3 - 5.0*p4 - p5)/(f + 5.0*m + 2.0*y - 6.0)';
    cv[5] = 'q4  = (-2.0*pow(qal,2.0) - 2.0*qal*f + qal*m + qal*p0 - 4.0*qal*p3 - 5.0*qal*p4 - qal*p5 - 2.0*qal*y + 2.0*qal + f*m + f*p0 - 4.0*f*p3 - 4.0*f*p4 - f*p5 + 5.0*m*p4 + m*y - m + p0*y - p0 - 4.0*p3*y + 4.0*p3 - 3.0*p4*y - p4 - p5*y + p5)/(qal*f + 5.0*qal*m + 2.0*qal*y - 6.0*qal + pow(f,2.0) + 5.0*f*m + 3.0*f*y - 7.0*f + 5.0*m*y - 5.0*m + 2.0*pow(y,2.0) - 8.0*y + 6.0)';
    cv[6] = 'q1  = (10.0*pow(qal,2.0) - 2.0*qal*f - 25.0*qal*m - 5.0*qal*p0 + 20.0*qal*p3 + 25.0*qal*p4 + 5.0*qal*p5 - 14.0*qal*y + 22.0*qal - 4.0*pow(f,2.0) - 21.0*f*m - f*p0 - 4.0*f*p1 + 4.0*f*p3 - 4.0*f*p4 + f*p5 - 12.0*f*y + 28.0*f - 20.0*m*p1 - 45.0*m*p4 - 17.0*m*y + 21.0*m + 3.0*p0*y + p0 - 8.0*p1*y + 24.0*p1 - 12.0*p3*y - 4.0*p3 - 33.0*p4*y + 49.0*p4 - 3.0*p5*y - p5 - 8.0*pow(y,2.0) + 32.0*y - 24.0)/(5.0*(-qal*f - 5.0*qal*m - 2.0*qal*y + 6.0*qal + f*m + f*y - f + 5.0*pow(m,2.0) + 7.0*m*y - 11.0*m + 2.0*pow(y,2.0) - 8.0*y + 6.0))';

    in_var   = ['xc','y','f','m','qal','q1','q4','log']   
    out_var  = ['x[0]','x[1]','x[2]','x[3]','x[4]','x[5]','x[6]','ln']

    in_sf   = ['xMgM1','xFeM1','xAlM1','xMgM23','xMnM23','xFeM23','xMgM4','xFeM4','xFe3M4','xAlM4','xSiT2','xAlT2'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]','sf[6]','sf[7]','sf[8]','sf[9]','sf[10]','sf[11]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def ctd_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 4

    W         = [0]*6
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.19;
    IG        = [0]*(n_em-1);
    cv        = [0]*(n_em-1);
    
    # Validity bounds
    Range     = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1];
    
    emname    = ['mctd','fctd','mnct','ctdo']
    make      = [0]*n_em
    make[0]   = sympify('mctd');
    make[1]   = sympify('fctd');
    make[2]   = sympify('mnctd + 0.66');
    make[3]   = sympify('mctd + 1/4 *andr - 1/4 *gr + 13.5');
    
    p[0] = sympify('1 - f - m - xc + m *xc');
    p[1] = sympify('xc + (-m) *xc');
    p[2] = sympify('m');
    p[3] = sympify('f');
    
    #     Site fractions
    xAlM1A = sympify('1 - f');
    xFe3M1A = sympify('f');
    xFeM1B = sympify('xc + (-m) *xc');
    xMgM1B = sympify('1 - m - xc + m *xc');
    xMnM1B = sympify('m');

    sf = [xAlM1A,xFe3M1A,xFeM1B,xMgM1B,xMnM1B];
  
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 4
    W[1] = 3
    W[2] = 1
    W[3] = 3
    W[4] = 5
    W[5] = 4
    v    = 0.
    
    idm[0] = sympify('xAlM1A**(1/2) *xMgM1B');
    idm[1] = sympify('xAlM1A**(1/2)* xFeM1B');
    idm[2] = sympify('xAlM1A**(1/2) *xMnM1B');
    idm[3] = sympify('xFe3M1A**(1/2)* xMgM1B');
    
    sym_sf = ['xAlM1A','xFe3M1A','xFeM1B','xMgM1B','xMnM1B'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    
    symb    = ['xc','m','f']
    
    cv[0]   = 'f  = p3';
    cv[1]   = 'm  = p2';
    cv[2]   = 'xc = p1/(1.0 - m)';

    in_var  = ['xc','m','f','log']   
    out_var = ['x[0]','x[1]','x[2]','ln']
    in_sf   = ['xAlM1A','xFe3M1A','xFeM1B','xMgM1B','xMnM1B'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def sp_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 4

    W         = [0]*6
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.124;
    IG       = [0]*(n_em-1);
    cv       = [0]*(n_em-1);
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1]; 
    
    emname    = ['herc','sp','mt','usp']
    make      = [0]*n_em
    make[0]   = sympify('herc');
    make[1]   = sympify('sp');
    make[2]   = sympify('mt');
    make[3]   = sympify('usp');
    
    p[0]  = sympify('y + (-1 + xc) *(1 + z)');
    p[1]  = sympify('(1 - xc)* (1 + z)');
    p[2]  = sympify('1 - y - z');
    p[3]  = sympify('z');


    #     Site fractions
    xAl = sympify('y');
    xFe3 = sympify('1 - y - z');
    xTi = sympify('z');
    xMg = sympify('1 - xc');
    xFe2 = sympify('xc');

    sf    = [xAl,xFe3,xTi,xMg,xFe2];
  
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 16
    W[1] = 2
    W[2] = 20
    W[3] = 18
    W[4] = 36
    W[5] = 30
    v    = 0.0

    idm[0] = sympify('xAl *xFe2');
    idm[1] = sympify('xAl *xMg');
    idm[2] = sympify('xFe3 *xFe2');
    idm[3] = sympify('xTi *xFe2');


    sym_sf = ['xAl','xFe3','xTi','xMg','xFe2'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');

    symb    = ['xc','y','z']
    
    cv[0]   = 'z  = p3';
    cv[1]   = 'y  = 1.0 - p2 - z';
    cv[2]   = 'xc = (-p1 + z + 1.0)/(z + 1.0)';

    in_var  = ['xc','y','z','log']   
    out_var = ['x[0]','x[1]','x[2]','ln']
    
    in_sf   = ['xAl','xFe3','xTi','xMg','xFe2'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def ilm_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 5

    W         = [0]*10
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.124;
    IG       = [0]*(n_em-1);
    cv       = [0]*(n_em-1);
   
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    Range[2]  = [0,1]; 
    Range[3]  = [-1,1];
    
    emname    = ['oilm','dilm','dhem','geik','pnt']
    make      = [0]*n_em
    make[0]   = sympify('d_ilm - 13.6075 + 0.009426 *T');
    make[1]   = sympify('d_ilm + 1.9928 - 0.0021 *T');
    make[2]   = sympify('d_hem');
    make[3]   = sympify('geik');
    make[4]   = sympify('pnt');

    p[0] = sympify('q');
    p[1] = sympify('-z + xc - m - q');
    p[2] = sympify('1 - xc');
    p[3] = sympify('z');
    p[4] = sympify('m');
    
    #     Site fractions
    xFeA = sympify('-1/2 *z + 1/2 *xc - 1/2 *m + 1/2* q');
    xTiA = sympify('-1/2 *z + 1/2 *xc - 1/2 *m - 1/2 *q');
    xMgA = sympify('z');
    xMnA = sympify('m');
    xFe3A = sympify('1 - xc');
#     xFeB = sympify('-1/2 *z + 1/2 *xc - 1/2 *m - 1/2 *q');
    xTiB = sympify('1/2 *z + 1/2 *xc + 1/2* m + 1/2 *q');
#     xFe3B = sympify('1 - xc');

    # last sf removed to avoid ill conditionned ineq
    sf    = [xFeA,xTiA,xMgA,xMnA,xFe3A,xTiB];
  
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 15.6
    W[1] = 26.6
    W[2] = 4
    W[3] = 2
    W[4] = 11
    W[5] = 4
    W[6] = 2
    W[7] = 36
    W[8] = 25
    W[9] = 4
    v    = 0.0

    idm[0] = sympify('xFeA *xTiB');
    idm[1] = sympify('4*xFeA**(1/2) *xTiA**(1/2)* xTiA**(1/2)* xTiB**(1/2)');
    idm[2] = sympify('xFe3A *xFe3A');
    idm[3] = sympify('xMgA* xTiB');
    idm[4] = sympify('xMnA* xTiB');
    
    sym_sf = ['xFeA','xTiA','xMgA','xMnA','xFe3A','xTiB'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');
    gbase[3]  = sympify('gb3');
    gbase[4]  = sympify('gb4');
    
    symb    = ['xc','z','m','q']
    
    cv[0]   = 'm  = p4';
    cv[1]   = 'z  = p3';
    cv[2]   = 'q  = p0';
    cv[3]   = 'xc  = 1.0 - p2';
 
    in_var  = ['xc','z','m','q','log']   
    out_var = ['x[0]','x[1]','x[2]','x[3]','ln']
    
    in_sf   = ['xFeA','xTiA','xMgA','xMnA','xFe3A','xTiB'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]','sf[5]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

def mt_mp():
    
    P = symbols('P')
    T = symbols('T')
    eps = symbols('eps')
    n_em = 3

    W         = [0]*3
    v         = [0]*n_em
    gbase     = [0]*n_em;
    p         = [0]*n_em;
    idm       = [0]*n_em;
    stp       = 0.19;
    # Initial guess -> ca,k
    IG       = [0]*(n_em-1);
    cv       = [0]*(n_em-1);
    
    # Validity bounds
    Range    = [[0]*2]*(n_em-1); # declare range

    Range[0]  = [0,1];
    Range[1]  = [0,1];
    
    emname    = ['imt','dmt','usp']
    make      = [0]*n_em
    make[0]   = sympify('d_mt - 1.8595 + 0.003166 *T');
    make[1]   = sympify('d_mt + 1.3305 - 0.001184 *T');
    make[2]   = sympify('usp');


    p[0] = sympify('-2*xc + 3*q');
    p[1] = sympify('3*xc - 3*q');
    p[2] = sympify('1 - xc');
    
    #     Site fractions
    xTiM  = sympify('1/2 - 1/2 *xc');
    xFe3M = sympify('xc - 1/2 *q');
    xFeM  = sympify('1/2 - 1/2 *xc + 1/2* q');
    xFe3T = sympify(' q');
    xFeT  = sympify('1 - q');
    
    sf = [xTiM,xFe3M,xFeM,xFe3T,xFeT];
  
    # Non-ideality by symmetric formalism
    symmetry = 1;

    W[0] = 2.4
    W[1] = 1
    W[2] = -5
    v    = 0.

    idm[0] = sympify('4*xFe3M *xFeM *xFe3T');
    idm[1] = sympify('27/4*xFe3M**(4/3) *xFeM**(2/3) *xFe3T**(2/3) *xFeT**(1/3)');
    idm[2] = sympify('4*xTiM *xFeM *xFeT');

    sym_sf =  ['xTiM','xFe3M','xFeM','xFe3T','xFeT'];
    
    #     Gbase must be initiated, then updated by each levelling
    gbase[0]  = sympify('gb0');
    gbase[1]  = sympify('gb1');
    gbase[2]  = sympify('gb2');

    symb    = ['xc','q']
    
    cv[0]   = 'xc  = 1.0 - p2';
    cv[1]   = 'q   = (3.0*xc - p1)/3.0';
 
    in_var  = ['xc','q','log']   
    out_var = ['x[0]','x[1]','ln']
    in_sf   = ['xTiM','xFe3M','xFeM','xFe3T','xFeT'];
    out_sf  = ['sf[0]','sf[1]','sf[2]','sf[3]','sf[4]']

    return symb, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname

op = ''

In [111]:
# CHOOSE PHASE
# bi g ep pl4T mu ol cd opx cpx spn hb liq fl ilm ilm_TJBH_2019
ss = 'sp_mp'

# output
db0  = ss[-2:];
ss0  = ss[:-3];

name = ss+'()'
sym_list, cv, p, v, W, gbase, sf, idm, stp, n_em, symmetry, Range, in_var, out_var, in_sf, out_sf, make, emname  = eval(name)


In [112]:
# CALCULATE f0, jac and Hessian

# save some data (needed later)
p_fct_cv  = [0]*n_em
for ii in range(0,n_em):
    p_fct_cv[ii] = p[ii]

R   = sympify("R");
T   = sympify("T");
eps = sympify("eps");

phi_p           = np.identity(n_em);
mu_Gex          = [0]*n_em;
mat_muMix       = [0]*n_em;
mu_Total        = [0]*n_em;
mat_phi         = [0]*n_em;
Gsol            = 0;
total_chem_comp = 0;

#declare mu0[ii] and p[ii] again here
muzx  = [0]*n_em
px    = [0]*n_em
for ii in range(0,n_em):
    px[ii]    = sympify('p'+str(ii))
    muzx[ii]  = sympify('muz'+str(ii))

if symmetry == 0:
    # Third step
    dp_dx = [[0] * (n_em-1) for i in range(n_em)]
    for ii in range(0,n_em):
        for jj in range(0,n_em-1):
            dp_dx[ii][jj] = diff(p[ii],sym_list[jj]) 
            
    # Second step            
    pfrac = p        
    p  = [0]*n_em
    px = [0]*n_em
    for ii in range(0,n_em):
        p[ii]  = sympify('p'+str(ii))
        px[ii] = sympify('p'+str(ii))
        
    sum_v   = sum(np.multiply(p,v)) 
    mat_phi = np.multiply(p,v)/(sum_v)
    mat_phix= nsimplify(mat_phi,tolerance=10**-12)

    mat_phi = [0]*n_em
    for ii in range(0,n_em):
        mat_phi[ii] = sympify('mat_phi'+str(ii))

    for ii in range(0,n_em):
        mu = 0;
        it = 0;
        for jj in range (0,n_em-1):
            for kk in range(jj+1,n_em):
                mu = mu - (phi_p[ii][jj] - mat_phi[jj])*(phi_p[ii][kk]-mat_phi[kk])*(W[it]*2*v[ii]/(v[jj]+v[kk]) );
                it = it + 1;
        if len(W) != it:
            display("Something is wrong with the mu model")
            
        mu_Gex[ii]      = nsimplify(mu,tolerance=10**-12); 

if symmetry == 1:
    mat_phix = []
    
    # Third step
    dp_dx = [[0] * (n_em-1) for i in range(n_em)]
    for ii in range(0,n_em):
        for jj in range(0,n_em-1):
            dp_dx[ii][jj] = diff(p[ii],sym_list[jj]) 

    pfrac = p       
    p  = [0]*n_em
    px = [0]*n_em
    
    for ii in range(0,n_em):
        p[ii]  = sympify('p'+str(ii))
        px[ii] = sympify('p'+str(ii))
        
    for ii in range(0,n_em):
        mu = 0;
        it = 0;
        for jj in range (0,n_em-1):
            for kk in range(jj+1,n_em):
                mu = mu - (phi_p[ii][jj] - p[jj])*(phi_p[ii][kk]-p[kk])*W[it];
                it = it + 1;
        if len(W) != it:
            display("Something is wrong with the mu model")
            
        mu_Gex[ii]      = mu; 
        Gsol            = Gsol + px[ii]*muzx[ii];

# nsimplify(mu,tolerance=10**-12)#        
##### GET MU_GEX #######
########################
mu_Gexx = [0]*n_em        
for ii in range(0,n_em):
    mu_Gexx[ii] = sympify('mu_Gex'+str(ii))
    
    
##### GET CHEMICAL POTENTIAL MU = g0 +RTlog(idm) +Gex #####
###########################################################
muz  = [0]*n_em
muz0 = [0]*n_em
for ii in range(0,n_em):
    muz[ii] = (gbase[ii] + mu_Gexx[ii] + R*T*log(idm[ii]))

for i,b in enumerate(muz):
    for a in preorder_traversal(b):
        if isinstance(a, Rational):
            muz[i] = muz[i].subs(a, float(a)) 
    
for ii in range(0,n_em):
    muz[ii] = ccode(muz[ii])
    
    
##### MAKE ENDMEMBERS #####  
########################### 
for i,b in enumerate(make):
    for a in preorder_traversal(b):
        if isinstance(a, Rational):
            make[i] = make[i].subs(a, float(a))    
    
    
##### SITE FRACTIONS #####  
##########################
sf_size   = len(sf)
sf0       = [0]*sf_size

for oo,ii in enumerate(sf):
    sf0[oo] = -1.0*sf[oo] 

for i,b in enumerate(sf):
    for a in preorder_traversal(b):
        if isinstance(a, Rational):
            sf[i] = sf[i].subs(a, float(a))
            
for i,b in enumerate(sf0):
    for a in preorder_traversal(b):
        if isinstance(a, Rational):
            sf0[i] = sf0[i].subs(a, float(a)) 
             
for ii, val in enumerate(sf0):  
    t0 = str(val)

    for i,val in enumerate(in_var):
        inv  = str(in_var[i])
        outv = str(out_var[i])
        t0 = replaceSymbols(t0,inv,outv) 
    sf0[ii] = t0   
    
    
##### SITE FRACTIONS FIRST DERIVATIVE ######
############################################
sym_list_size = len(sym_list)
ineq_sf = [['' for xx in range(sym_list_size)] for xx in range(sf_size)]

for oo,ii in enumerate(sf):
    for pi,jj in enumerate(sym_list):
        ineq_sf[oo][pi]  = diff(ii,jj);  
        ineq_sf[oo][pi] *= -1.0;
        
for i,b in enumerate(ineq_sf):
    for j,c in enumerate(b):
        for a in preorder_traversal(c):
            if isinstance(a, Rational):
                ineq_sf[i][j] = ineq_sf[i][j].subs(a, float(a))
                
for ii, val in enumerate(ineq_sf):
    for jj, val0 in enumerate(val):
        t0 = str(val0)
        for i,val in enumerate(in_var):
            inv  = str(in_var[i])
            outv = str(out_var[i])
            t0 = replaceSymbols(t0,inv,outv) 

        ineq_sf[ii][jj] = t0


##### IDEAL MIXING TERM (idm) ######
####################################
for i,b in enumerate(idm):
    for a in preorder_traversal(b):
        if isinstance(a, Rational):
            idm[i] = idm[i].subs(a, float(a))
    
##### EM FRACTION ######
######################## 
for i,b in enumerate(pfrac):
    for a in preorder_traversal(b):
        if isinstance(a, Rational):
            pfrac[i] = pfrac[i].subs(a, float(a))

            
##### EM FRACTION FIRST DERIVATIVE ######
#########################################     
for i,b in enumerate(dp_dx):
    for j,c in enumerate(b):
        for a in preorder_traversal(c):
            if isinstance(a, Rational):    
#                 dp_dx[i][j] = nsimplify(dp_dx[i][j],tolerance=10**-12)
                dp_dx[i][j] = dp_dx[i][j].subs(a, float(a))



 

In [113]:
# OBJECTIVE FUNCTION (obj_)
if 0==0:
    pout = [0]*(len(sf))

    for ii, val in enumerate(sf):
        t0 = str(val)
        for i,val in enumerate(in_var):
            inv  = str(in_var[i])
            outv = str(out_var[i])
            t0 = replaceSymbols(t0,inv,outv) 

        pout[ii] = t0
    
    pout2 = muz0

    for ii, val in enumerate(muz):
        t0 = str(val)
        for i,val in enumerate(in_sf):
            inv  = str(in_sf[i])
            outv = str(out_sf[i])
            t0 = replaceSymbols(t0,inv,outv) 

        pout2[ii] = t0
    
    op = ''
    os = '    '
    op += os+'\n' 
    op += '/**\n'
    op += os + 'Objective function of '+ss+'\n'
    op += '*/\n'
    op += 'double obj_'+db0+'_'+ss0+'(unsigned n, const double *x, double *grad, void *SS_ref_db){\n'
    op += os+'SS_ref *d         = (SS_ref *) SS_ref_db;\n\n'
    op += os+'int n_em          = d->n_em;\n'
    op += os+'double P          = d->P;\n'
    op += os+'double T          = d->T;\n'
    op += os+'double R          = d->R;\n\n'

    op += os+'double *gb        = d->gb_lvl;\n'
    if (symmetry == 0):
        op += os+'double *mat_phi   = d->mat_phi;\n'
    op += os+'double *mu_Gex    = d->mu_Gex;\n'
    op += os+'double *sf        = d->sf;\n'
    op += os+'double *mu        = d->mu;\n'

    op += os+'px_'+db0+'_'+ss0+'(SS_ref_db,x);\n\n'
    
    if (symmetry == 1):
        op += os+'for (int i = 0; i < n_em; i++){\n'
        op += os+os+'mu_Gex[i] = 0.0;\n'
        op += os+os+'int it    = 0;\n'
        op += os+os+'for (int j = 0; j < d->n_xeos; j++){\n'
        op += os+os+os+'for (int k = j+1; k < n_em; k++){\n'
        op += os+os+os+os+'mu_Gex[i] -= (d->eye[i][j] - d->p[j])*(d->eye[i][k] - d->p[k])*(d->W[it]);\n'  
        op += os+os+os+os+'it += 1;\n'  
        op += os+os+os+'}\n'  
        op += os+os+'}\n'   
        op += os+'}\n'
        
    if (symmetry == 0):
        op += os+'d->sum_v = 0.0;\n'
        op += os+'for (int i = 0; i < n_em; i++){\n'     
        op += os+os+'d->sum_v += d->p[i]*d->v[i];\n'     
        op += os+'}\n'
        op += os+'for (int i = 0; i < n_em; i++){\n'
        op += os+os+'d->mat_phi[i] = (d->p[i]*d->v[i])/d->sum_v;\n'
        op += os+'}\n'
        op += os+'\n'
        op += os+'for (int i = 0; i < d->n_em; i++){\n' 
        op += os+os+'mu_Gex[i] = 0.0;\n'
        op += os+os+'int it = 0;\n'
        op += os+os+'for (int j = 0; j < d->n_xeos; j++){\n'
        op += os+os+os+'for (int k = j+1; k < d->n_em; k++){\n'    
        op += os+os+os+os+'mu_Gex[i] -= (d->eye[i][j] - d->mat_phi[j])*(d->eye[i][k] - d->mat_phi[k])*(d->W[it]*2.0*d->v[i]/(d->v[j]+d->v[k]));\n'    
        op += os+os+os+os+'it += 1;\n'
        op += os+os+os+'}\n'
        op += os+os+'}\n'
        op += os+'}\n'
        
    op += os+'\n'
    for  i,val in enumerate(sf):
        op += os+'sf['+str(i)+']          = '+((pout[i]))+';\n'    
    op += os+'\n'
    op += os+'\n'
    for  i,val in enumerate(muz):
        op += os+'mu['+str(i)+']          = '+(digit2index2(pout2[i]))+';\n'    
    op += os+'\n'

    op += os+'d->sum_apep = 0.0;\n'
    op += os+'for (int i = 0; i < n_em; i++){\n'
    op += os+os+'d->sum_apep += d->ape[i]*d->p[i];\n'
    op += os+'}\n'
    op += os+'d->factor = d->fbc/d->sum_apep;\n\n'

    op += os+'d->df_raw = 0.0;\n'
    op += os+'for (int i = 0; i < n_em; i++){\n'
    op += os+os+'d->df_raw += mu[i]*d->p[i];\n'
    op += os+'}\n'
    op += os+'d->df = d->df_raw * d->factor;\n\n'

    op += os+'if (grad){\n'
    op += os+os+'double *dfx    = d->dfx;\n'
    op += os+os+'double **dp_dx = d->dp_dx;\n'
    op += os+os+'dpdx_'+db0+'_'+ss0+'(SS_ref_db,x);\n'
    op += os+os+'for (int i = 0; i < (d->n_xeos); i++){\n'
    op += os+os+os+'dfx[i] = 0.0;\n'
    op += os+os+os+'for (int j = 0; j < n_em; j++){\n'
    op += os+os+os+os+'dfx[i] += (mu[j] - (d->ape[j]/d->sum_apep)*d->df_raw)*d->factor*dp_dx[j][i];\n'
    op += os+os+os+'}\n'
    op += os+os+os+'grad[i] = creal(dfx[i]);\n'
    op += os+os+'}\n'
    op += os+'}\n\n'
    op += os+'return d->df;\n' 
    op += '}\n'
    
print(op)

    
/**
    Objective function of sp_mp
*/
double obj_mp_sp(unsigned n, const double *x, double *grad, void *SS_ref_db){
    SS_ref *d         = (SS_ref *) SS_ref_db;

    int n_em          = d->n_em;
    double P          = d->P;
    double T          = d->T;
    double R          = d->R;

    double *gb        = d->gb_lvl;
    double *mu_Gex    = d->mu_Gex;
    double *sf        = d->sf;
    double *mu        = d->mu;
    px_mp_sp(SS_ref_db,x);

    for (int i = 0; i < n_em; i++){
        mu_Gex[i] = 0.0;
        int it    = 0;
        for (int j = 0; j < d->n_xeos; j++){
            for (int k = j+1; k < n_em; k++){
                mu_Gex[i] -= (d->eye[i][j] - d->p[j])*(d->eye[i][k] - d->p[k])*(d->W[it]);
                it += 1;
            }
        }
    }
    
    sf[0]          = x[1];
    sf[1]          = -1.0*x[1] - 1.0*x[2] + 1.0;
    sf[2]          = x[2];
    sf[3]          = 1.0 - 1.0*x[0];
    sf[4]          = x[0];
    
    
    mu[0]          = R*T*creal(clog(sf[0]*sf

In [114]:
# ENDMEMBER FRACTION DERIVATIVE (dpdx)
if 0==0:
    pout = (dp_dx)
    for ii, val in enumerate(dp_dx):
        for jj, val0 in enumerate(val):
            t0 = (str((val0)))
            for i,val in enumerate(in_var):
                inv  = str(in_var[i])
                outv = str(out_var[i])
                t0 = replaceSymbols(t0,inv,outv) 

            pout[ii][jj] = t0


#     op = ''
    os = '    '
    op = ''
    op += '\n' 
    op += '/**\n'
    op += os + 'Update dpdx matrix of '+ss+'\n'
    op += '*/\n'
    op += 'void dpdx_'+db0+'_'+ss0+'(void *SS_ref_db, const double *x){\n'
    op += os+'SS_ref *d  = (SS_ref *) SS_ref_db;\n'
    op += os+'double **dp_dx = d->dp_dx;\n\n'

    for  i,val in enumerate(pout):
        for j,val2 in enumerate(val):
            op += os+'dp_dx['+str(i)+']['+str(j)+'] = '+((pout[i][j]))+';  '  
        op += os+'\n'
    op += '}\n'
    
print(op)


/**
    Update dpdx matrix of sp_mp
*/
void dpdx_mp_sp(void *SS_ref_db, const double *x){
    SS_ref *d  = (SS_ref *) SS_ref_db;
    double **dp_dx = d->dp_dx;

    dp_dx[0][0] = x[2] + 1.0;      dp_dx[0][1] = 1.00000000000000;      dp_dx[0][2] = x[0] - 1.0;      
    dp_dx[1][0] = -1.0*x[2] - 1.0;      dp_dx[1][1] = 0.0;      dp_dx[1][2] = 1.0 - 1.0*x[0];      
    dp_dx[2][0] = 0.0;      dp_dx[2][1] = -1.00000000000000;      dp_dx[2][2] = -1.00000000000000;      
    dp_dx[3][0] = 0.0;      dp_dx[3][1] = 0.0;      dp_dx[3][2] = 1.00000000000000;      
}



In [115]:
# INEQUALITY CONSTRAINTS (sf)
if 0==0:
    out_name    = 'MAGEMin_Ineq_'+db0+'_'+ss0+".txt"
    text_file   = open(out_name, "w")
    op = ''
    os = '    '
    
    op += '/**\n'
    op += os + 'Inequality constraints for '+ss+'\n'
    op += '*/\n'
    op += 'void '+ss+'_c(unsigned m, double *result, unsigned n, const double *x, double *grad, void *data){\n'
    for ii, val0 in enumerate(ineq_sf):
#         op += os+'double c'+ str(ii)+'(unsigned n, const double *x, double *grad, void *data){\n'   
        op += os+'result['+str(ii)+'] = ( eps_sf + ' + str(sf0[ii])+');\n' 
    op += '\n' 
    op += os+'if (grad) {\n' 
    n = 0;
    for ii, val0 in enumerate(ineq_sf):
        for jj, val1 in enumerate(val0):
            op += os+os+'grad['+str(n)+'] = ' + str((val1)) + ';\n' 
            n += 1
    op += os+'}\n' 
    
    op += '\n'  
    op += os+'return;\n'    
    op += '};\n' 
    op += '\n'  
    
    print("save Ineq list to file   :",out_name, "...")
    n         = text_file.write(op)
    print(op)

save Ineq list to file   : MAGEMin_Ineq_mp_sp.txt ...
/**
    Inequality constraints for sp_mp
*/
void sp_mp_c(unsigned m, double *result, unsigned n, const double *x, double *grad, void *data){
    result[0] = ( eps_sf + -1.0*x[1]);
    result[1] = ( eps_sf + 1.0*x[1] + 1.0*x[2] - 1.0);
    result[2] = ( eps_sf + -1.0*x[2]);
    result[3] = ( eps_sf + 1.0*x[0] - 1.0);
    result[4] = ( eps_sf + -1.0*x[0]);

    if (grad) {
        grad[0] = 0.0;
        grad[1] = -1.00000000000000;
        grad[2] = 0.0;
        grad[3] = 0.0;
        grad[4] = 1.00000000000000;
        grad[5] = 1.00000000000000;
        grad[6] = 0.0;
        grad[7] = 0.0;
        grad[8] = -1.00000000000000;
        grad[9] = 1.00000000000000;
        grad[10] = 0.0;
        grad[11] = 0.0;
        grad[12] = -1.00000000000000;
        grad[13] = 0.0;
        grad[14] = 0.0;
    }

    return;
};




In [116]:
# ENDMEMBER FRACTION (px)
if 0==0:
    pout = p

    for ii, val in enumerate(pfrac):
        t0 = str(val)
        for i,val in enumerate(in_var):
            inv  = str(in_var[i])
            outv = str(out_var[i])
            t0 = replaceSymbols(t0,inv,outv) 

        pout[ii] = t0


    op = ''
    os = '    '
    op += os+'\n' 
    op += '/**\n'
    op += os + 'Endmember fraction of '+ss+'\n'
    op += '*/\n'
    op += 'void px_'+db0+'_'+ss0+'(void *SS_ref_db, const double *x){\n'
    op += os+'SS_ref *d  = (SS_ref *) SS_ref_db;\n'
    op += os+'double *p = d->p;\n'
    
    for  i,val in enumerate(pfrac):
        op += os+'    p['+str(i)+']           = '+((pout[i]))+';\n'    
    op += '}\n'
    
print(op)

    
/**
    Endmember fraction of sp_mp
*/
void px_mp_sp(void *SS_ref_db, const double *x){
    SS_ref *d  = (SS_ref *) SS_ref_db;
    double *p = d->p;
        p[0]           = x[1] + (x[0] - 1.0)*(x[2] + 1.0);
        p[1]           = (1.0 - 1.0*x[0])*(x[2] + 1.0);
        p[2]           = -1.0*x[1] - 1.0*x[2] + 1.0;
        p[3]           = x[2];
}



In [117]:
# MAKE ENDMEMBERS
# if 0==0:


exclude = [sympify('P'),sympify('T')];
em = [];
for i in make:
    for key in i.free_symbols:
        if key not in em and key not in exclude:
            em.append(key)

            
out_name    = 'MAGEMin_make_'+db0+'_'+ss0+".txt"
text_file   = open(out_name, "w")          
            
op = ''
os = '    '

  

op += '/**\n'
op += '   retrieve reference thermodynamic data for '+db0+'_'+ss0+'\n'
op += '*/\n'
op += 'SS_ref G_SS_'+db0+'_'+ss0+'_function(SS_ref SS_ref_db, int EM_database, int len_ox, bulk_info z_b, double eps){\n'
op += os+'\n'
op += os+'int i, j;\n'
op += os+'int n_em = SS_ref_db.n_em;\n'
op += os+'\n'
op += os+'char   *EM_tmp[] 		= {'
for i,val in enumerate(emname):
    if i < n_em-1:
        op += '"'+str(val)+'",'
    else:
        op += '"'+str(val)+'"'
op += '};\n'
op += os+'for (int i = 0; i < SS_ref_db.n_em; i++){\n'
op += os+os+'strcpy(SS_ref_db.EM_list[i],EM_tmp[i]);\n'
op += os+'};\n'

op += os+'\n'
for i in range(0,len(W)):
    op += os+'SS_ref_db.W['+str(i)+'] = '+print_W(W[i])+';\n'
op += os+'\n'
if (type(v) != float):
    for i in range(0,len(v)):
        op += os+'SS_ref_db.v['+str(i)+'] = '+str(v[i])+';\n'
    op += os+'\n'
op += os+'\n'
for i in em:
    if (str(i)[0:2] == 'd_'):
        opt = 'disordered'
        suf = 'di';
        name = str(i)[2:] 
    elif (str(i)[0:2] == 'o_'):
        opt = 'ordered'
        suf = 'or';
        name = str(i)[2:] 
    elif (str(i)[0:2] == 'e_'):
        opt = 'equilibrium'
        suf = 'eq';
        name = str(i)[2:] 
    else:
        opt = 'equilibrium'
        suf = 'eq';
        name = str(i)
        
    op += os+'em_data '+name+'_'+suf+' 		= get_em_data(		EM_database, \n'
    op += os+'										len_ox,\n'
    op += os+'										z_b,\n'
    op += os+'										SS_ref_db.P,\n'
    op += os+'										SS_ref_db.T,\n'
    op += os+'										"'+name+'", \n'
    op += os+'										"'+opt+'"	);\n'
    op += os+'\n'
    
for i in range(0,n_em):
    op += os+ 'SS_ref_db.gbase['+str(i)+'] 		= '+print_em_gb(em,make[i])+';\n'
op += os+'\n'

for i in range(0,n_em):
    op += os+ 'SS_ref_db.ElShearMod['+str(i)+'] 	= '+print_em_shearM(em,make[i])+';\n'
op += os+'\n'

op += os+'for (i = 0; i < len_ox; i++){\n'
for i in range(0,n_em):
    op += os+os+ 'SS_ref_db.Comp['+str(i)+'][i] 	= '+print_em_C(em,make[i])+';\n'
op += os+'}\n'
op += os+'\n'
op += os+'for (i = 0; i < n_em; i++){\n'
op += os+os+'SS_ref_db.z_em[i] = 1.0;\n'
op += os+'};\n'
op += os+'\n'
for i in range(0,n_em-1):
    op += os+ 'SS_ref_db.bounds_ref['+str(i)+'][0] = '+str(float(Range[i][0]))+'+eps;  SS_ref_db.bounds_ref['+str(i)+'][1] = '+str(float(Range[i][1]))+'-eps;\n'
op += os+'\n'
op += os+'return SS_ref_db;\n'
op += '}\n'
print(op)

print("save make to file   :",out_name, "...")
n         = text_file.write(op)



/**
   retrieve reference thermodynamic data for mp_sp
*/
SS_ref G_SS_mp_sp_function(SS_ref SS_ref_db, int EM_database, int len_ox, bulk_info z_b, double eps){
    
    int i, j;
    int n_em = SS_ref_db.n_em;
    
    char   *EM_tmp[] 		= {"herc","sp","mt","usp"};
    for (int i = 0; i < SS_ref_db.n_em; i++){
        strcpy(SS_ref_db.EM_list[i],EM_tmp[i]);
    };
    
    SS_ref_db.W[0] = 16;
    SS_ref_db.W[1] = 2;
    SS_ref_db.W[2] = 20;
    SS_ref_db.W[3] = 18;
    SS_ref_db.W[4] = 36;
    SS_ref_db.W[5] = 30;
    
    
    em_data herc_eq 		= get_em_data(		EM_database, 
    										len_ox,
    										z_b,
    										SS_ref_db.P,
    										SS_ref_db.T,
    										"herc", 
    										"equilibrium"	);
    
    em_data sp_eq 		= get_em_data(		EM_database, 
    										len_ox,
    										z_b,
    										SS_ref_db.P,
    										SS_ref_db.T,
    										"sp", 
    										"equilibrium"	);
    
    em_data mt_eq 		= get_em_data(		EM_database, 
    							

In [118]:
# SOLUTION PHASE MEMORY SIZE
if 0==0:
        
    op = ''
    os = '    '
    
    op += '/**\n'
    op += os+'allocate memory for '+ss+'\n'
    op += '*/\n'
    op += 'SS_ref G_SS_'+db0+'_'+ss0+'_init_function(SS_ref SS_ref_db, int EM_database, global_variable gv){\n'
    op += os+'\n'
    op += os+'SS_ref_db.is_liq    = '+'0'+';\n' 
    op += os+'SS_ref_db.symmetry  = '+str(symmetry)+';\n' 
    op += os+'SS_ref_db.n_sf      = '+str(len(sf))+';\n' 
    op += os+'SS_ref_db.n_em      = '+str(len(p))+';\n' 
    if symmetry == 0:
        op += os+'SS_ref_db.n_v       = '+str(len(v))+';\n' 
    op += os+'SS_ref_db.n_w       = '+str(len(W))+';\n' 
    op += os+'SS_ref_db.n_xeos    = '+str(len(p)-1)+';\n' 
    op += os+'SS_ref_db.CstFactor = 0;\n' 
    op += os+'\n'
    op += os+'return SS_ref_db;\n' 
    op += '}\n'  
    
print(op)

/**
    allocate memory for sp_mp
*/
SS_ref G_SS_mp_sp_init_function(SS_ref SS_ref_db, int EM_database, global_variable gv){
    
    SS_ref_db.is_liq    = 0;
    SS_ref_db.symmetry  = 1;
    SS_ref_db.n_sf      = 5;
    SS_ref_db.n_em      = 4;
    SS_ref_db.n_w       = 6;
    SS_ref_db.n_xeos    = 3;
    SS_ref_db.CstFactor = 0;
    
    return SS_ref_db;
}



In [119]:
# NLopt_opt_function
if 0==0:
    

    
    op = ''
    os = '    '
    
    out_name    = 'MAGEMin_NLopt_'+db0+'_'+ss0+".txt"
    text_file   = open(out_name, "w")
    
    op += 'SS_ref NLopt_opt_'+db0+'_'+ss0+'_function(global_variable gv, SS_ref SS_ref_db){\n'
    op += os+'\n'
    op += os+'int    n_em     = SS_ref_db.n_em;\n'
    op += os+'unsigned int n  = SS_ref_db.n_xeos;\n'
    op += os+'unsigned int m  = SS_ref_db.n_sf;\n'
    op += os+'\n'
    op += os+'double *x  = SS_ref_db.iguess; \n'
    op += os+'\n'
    op += os+'for (int i = 0; i < (SS_ref_db.n_xeos); i++){\n'
    op += os+'   SS_ref_db.lb[i] = SS_ref_db.bounds[i][0];\n'
    op += os+'   SS_ref_db.ub[i] = SS_ref_db.bounds[i][1];\n'
    op += os+'}\n'
    op += os+'\n'
    op += os+'SS_ref_db.opt = nlopt_create(NLOPT_LD_CCSAQ, (n)); \n'
    op += os+'nlopt_set_lower_bounds(SS_ref_db.opt, SS_ref_db.lb);\n'
    op += os+'nlopt_set_upper_bounds(SS_ref_db.opt, SS_ref_db.ub);\n'
    op += os+'nlopt_set_min_objective(SS_ref_db.opt, obj_'+db0+'_'+ss0+', &SS_ref_db);\n'
    op += os+'nlopt_add_inequality_mconstraint(SS_ref_db.opt, m, '+ss0+'_'+db0+'_c, NULL, SS_ref_db.tol_sf);\n'
    op += os+'nlopt_set_ftol_rel(SS_ref_db.opt, gv.obj_tol);\n'
    op += os+'nlopt_set_maxeval(SS_ref_db.opt, gv.maxeval);\n'
    op += os+'\n'
    op += os+'double minf;\n'
    op += os+'if (gv.maxeval==1){  \n'
    op += os+'   // we are only interested in evaluating the objective function  \n'
    op += os+'   minf = obj_'+db0+'_'+ss0+'(n, x, NULL, &SS_ref_db);\n'
    op += os+'}\n'
    op += os+'else{\n'
    op += os+'  // do optimization\n'
    op += os+'  SS_ref_db.status = nlopt_optimize(SS_ref_db.opt, x, &minf);\n'
    op += os+'}\n'

    op += os+'/* Send back needed local solution parameters */\n'
    op += os+'for (int i = 0; i < SS_ref_db.n_xeos; i++){\n'
    op += os+'   SS_ref_db.xeos[i] = x[i];\n'
    op += os+'}\n'
   
    op += os+'\n'
    op += os+'SS_ref_db.df   = minf;\n'
    op += os+'nlopt_destroy(SS_ref_db.opt);\n' 
    
    op += os+'\n'
    op += os+'return SS_ref_db;\n' 
    op += '};\n'  
    
    print(op)
    
    print("save PC list to file   :",out_name, "...")
    n         = text_file.write(op)

SS_ref NLopt_opt_mp_sp_function(global_variable gv, SS_ref SS_ref_db){
    
    int    n_em     = SS_ref_db.n_em;
    unsigned int n  = SS_ref_db.n_xeos;
    unsigned int m  = SS_ref_db.n_sf;
    
    double *x  = SS_ref_db.iguess; 
    
    for (int i = 0; i < (SS_ref_db.n_xeos); i++){
       SS_ref_db.lb[i] = SS_ref_db.bounds[i][0];
       SS_ref_db.ub[i] = SS_ref_db.bounds[i][1];
    }
    
    SS_ref_db.opt = nlopt_create(NLOPT_LD_CCSAQ, (n)); 
    nlopt_set_lower_bounds(SS_ref_db.opt, SS_ref_db.lb);
    nlopt_set_upper_bounds(SS_ref_db.opt, SS_ref_db.ub);
    nlopt_set_min_objective(SS_ref_db.opt, obj_mp_sp, &SS_ref_db);
    nlopt_add_inequality_mconstraint(SS_ref_db.opt, m, sp_mp_c, NULL, SS_ref_db.tol_sf);
    nlopt_set_ftol_rel(SS_ref_db.opt, gv.obj_tol);
    nlopt_set_maxeval(SS_ref_db.opt, gv.maxeval);
    
    double minf;
    if (gv.maxeval==1){  
       // we are only interested in evaluating the objective function  
       minf = obj_mp_sp(n, x, NULL, &SS_ref_db);
    }
 

In [120]:
# Generate pseudocompounds
if 0==0:
    cv2sf = sf0

    sf_pc = [0.0]*len(sf0)
    for ii, val in enumerate(sf):
        t0 = str(val)
        for i,val in enumerate(in_var):
            inv  = str(in_var[i])
            outv = str(out_var[i])
            t0 = replaceSymbols(t0,inv,outv) 

        cv2sf[ii] = t0
        
        
    # you should not have the need to change the following section
    ini_name    = db0+'_'+ss0+'_ini'+'(shift)';
    run_name    = db0+'_'+ss0+'(x)';
    out_name    = 'MAGEMin_'+db0+'_'+ss0+"_PC.txt"
    shift       = 0.0001;
    d           = n_em-1;
#     stp         = 0.249    
    print("Database               : [",db0,"]")
    print("Solution phase         : [",ss0,"]")
    print("Dims                   : [",d,"]")
    print("step length            : [",stp,"]") 
 
    # generalized nested loop
    xeos_list   = [];
    min_val     = 0;
    ip          = [min_val]*(d+1);
    nstep       = [min_val]*(d+1);
    min_arr     = [min_val]*(d+1);
    max_arr     = [min_val]*(d+1);

    for i in range(0,d):
        nstep[i+1]   = int((Range[i][1] -Range[i][0])/stp)+1;
        min_arr[i+1] = Range[i][0];
        max_arr[i+1] = nstep[i+1];

    print("number of subdivisions :",nstep[1:]) 
    t = time.time()
    xeos_list   = [];
    pp           = d
    while ip[0] == min_val:

        x       = np.array(ip[1:d+1], dtype=np.float32)*stp+shift
        x      += np.array(min_arr[1:d+1], dtype=np.float32)
        sf_ok   = eval_sf(sf_pc,cv2sf)

        if sf_ok == 1:
            xeos_list.append(x)

        ip[pp] += 1;
        while ip[pp] == max_arr[pp]:
            ip[pp] = min_val;
            pp -= 1;
            ip[pp] += 1;
            if (ip[pp] != nstep[pp]):
                pp = d    
    elapsed = time.time() - t

    # save pseudocompound list to file             
    text_file   = open(out_name, "w")
    out   = np.array(xeos_list)
    npc   = len(out);
    print("time to generate PC    :",elapsed, "seconds")
    print("number of generated PC :",npc)
    
    nxeos = d;
    pp    = 'struct ss_pc '+db0+'_'+ss0+'_pc_xeos['+str(npc)+'] = {\n';
    for i in range(0,npc):
        pp += '{{'
        for j in range(0,nxeos):
            pp += str('%1.6f' %out[i][j])
            if j < nxeos-1:
                pp += (',')
            else:
                if i < npc-1:
                    pp += '}},'
                else:
                    pp += '}}};'
        pp += '\n'
    
    print("save PC list to file   :",out_name, "...")
    n         = text_file.write(pp)
    

Database               : [ mp ]
Solution phase         : [ sp ]
Dims                   : [ 3 ]
step length            : [ 0.124 ]
number of subdivisions : [9, 9, 9]
time to generate PC    : 0.05004596710205078 seconds
number of generated PC : 405
save PC list to file   : MAGEMin_mp_sp_PC.txt ...


In [121]:
test_p2x = 1;
# Generate P2X function
if 1 == 1:
    pout   = [0]*(n_em-1)
    iguess = [0]*(n_em-1)

    for ii, val in enumerate(cv):
        t0 = str(val)
        for i in range(0,n_em):
            inv  = 'p'+str(i)
            outv = 'd->p['+str(i)+']'
            t0   = replaceSymbols(t0,inv,outv) 

        for i in range(0,n_em-1):
            inv  = in_var[i]
            outv = 'd->iguess['+str(i)+']'
            t0   = replaceSymbols(t0,inv,outv) 

        pout[ii] = t0
    out_name    = 'MAGEMin_'+db0+'_'+ss0+"_p2x.txt"
    text_file   = open(out_name, "w")
    op = ''
    os = '    '
    op += os+'\n' 
    op += '/**\n'
    op += os + 'Endmember to xeos for '+ss+'\n'
    op += '*/\n'
    op += 'void p2x_'+db0+'_'+ss0+'(void *SS_ref_db, double eps){\n'
    op += os+'SS_ref *d  = (SS_ref *) SS_ref_db;\n'
    op += os+'\n' 
    for i in pout:
        op += os+str(i)+';\n'
        
    op += os+'\n' 
    op += os+'for (int i = 0; i < d->n_xeos; i++){\n' 
    op += os+os+'if (d->iguess[i] < d->bounds[i][0]){\n' 
    op += os+os+os+'d->iguess[i] = d->bounds[i][0];\n' 
    op += os+os+'}\n' 
    op += os+os+'if (d->iguess[i] > d->bounds[i][1]){\n' 
    op += os+os+os+'d->iguess[i] = d->bounds[i][1];\n' 
    op += os+os+'}\n' 
    op += os+'}\n' 
    op += os+'\n' 
    op += '}\n'
    

    print("save p2x to file   :",out_name, "...")
    n         = text_file.write(op)
    
    
    print(op)
if test_p2x == 1:
    pt   = [0]*(n_em-1)
    ig   = [0]*(n_em-1)

    for ii, val in enumerate(cv):
        t0 = str(val)
        for i in range(0,n_em):
            inv  = 'p'+str(i)
            outv = 'po['+str(i)+']'
            t0   = replaceSymbols(t0,inv,outv) 

        for i in range(0,n_em-1):
            inv  = in_var[i]
            outv = 'ig['+str(i)+']'
            t0   = replaceSymbols(t0,inv,outv) 

        pt[ii] = t0

    xeos = list(xeos_list[random.randrange(0, len(xeos_list))])
    po   = [0.0]*n_em;
#     print('cv_in ',xeos) 
    for i in range(0,n_em-1):
        exec ( in_var[i] +'='+ str(xeos[i]) )

    for i in range(0,n_em):
        po[i] = (eval(str(p_fct_cv[i])))

#     print('p     ',po)

    for i in pt:
        exec(i)

#     print('cv_p2x',ig) 
#     print('')
    norm = np.linalg.norm(np.array(ig)-np.array(xeos));
    print('\nTest computes the endmember fraction from random feasible cv (compositional variables), ')
    print('then the corresponding endmember fraction are used to compute cv with the p2x function.\n')
    print('Error norm (cv_in vs cv_out): [',norm,']')
    if norm < 1e-7:
        print('random test passed (max norm tol = 1e-7), function can be used in MAGEMin')
    else:
        print('random test failled, correct p2x formulation')

save p2x to file   : MAGEMin_mp_sp_p2x.txt ...
    
/**
    Endmember to xeos for sp_mp
*/
void p2x_mp_sp(void *SS_ref_db, double eps){
    SS_ref *d  = (SS_ref *) SS_ref_db;
    
    d->iguess[2]  = d->p[3];
    d->iguess[1]  = 1.0 - d->p[2] - d->iguess[2];
    d->iguess[0] = (-d->p[1] + d->iguess[2] + 1.0)/(d->iguess[2] + 1.0);
    
    for (int i = 0; i < d->n_xeos; i++){
        if (d->iguess[i] < d->bounds[i][0]){
            d->iguess[i] = d->bounds[i][0];
        }
        if (d->iguess[i] > d->bounds[i][1]){
            d->iguess[i] = d->bounds[i][1];
        }
    }
    
}


Test computes the endmember fraction from random feasible cv (compositional variables), 
then the corresponding endmember fraction are used to compute cv with the p2x function.

Error norm (cv_in vs cv_out): [ 2.8277207466080897e-08 ]
random test passed (max norm tol = 1e-7), function can be used in MAGEMin
